In [14]:
# DASHBOARD JEMBATAN TIMBANG SURVEYOR INDONESIA

# requirements:
# pastikan anda menguasai bidang ilmu berikut :
# statistics, machine learning, programming (python & sql)

# dashboard ini memiliki 3 halaman: 
# 1. rekapitulasi.
# 2. Forecasting
# 3. Rapor

# catatan :
# 1. code ini memakai python versi 3.9, ini adalah versi python terstabil, library yg ada sudah terintegrasi antara satu dengan yg lainnya. tidak disarankan menggunakan python versi terbaru.
# 2. code ini diflatten agar memudahkan eksekusi dan ringkas (sebelum diflatten terdapat 4300-an baris), partisikan berdasarkan sumber data, dash component, dan proses deployment agar mudah memahaminya.
# 3. sesuaikan data wrangling dan query database sesuai kebutuhan.
# 4. terdapat hardcode yang perlu diupdate secara manual, lakukan otomatisasi jika memungkinkan.
# 5. masukkan source data yang dibutuhkan. cari dengan cara ctrl/command + f ,kata kunci = "PATH_SOUCE_DATA".
# 6. masukkan attribute database. cari dengan cara ctrl/command + f, kata kunci = # DATABASE_ATTRIBUTE
# 7. Selalu update UPPKB mana yang mandiri dan pendampingan di query sql dibawah, karena tiap tahun selalu berubah.

# rekomendasi pengembangan fitur selanjutnya: 
# 1. hosting ke server
# 2. buat fitur download data, hubungkan ke goodgle drive SI. 
# 3. buat statistical test untuk data categorical (Chi Square statistical test)
# 4. tampilkan graph Mutual Information atau Recursive Feature Elimintaion untuk kebutuhan analisis independent variable dengan target variable
# 5. buat plot animasi bergerak dari plotly, seperti Bar Race, dll

# hanya satu orang yang mengerjakan project ini, mohon maaf jika dokumentasi petunjuk dan code kurang jelas
# terakhir... be happy!

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None) 
import matplotlib.pyplot as plt
import matplotlib as mpl 
import seaborn as sns
%matplotlib inline
import re
import math 
from dash import html, dcc, Dash, State, callback_context
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc 
from dash_bootstrap_templates import load_figure_template
import mysql.connector  
import sqlalchemy 
from sqlalchemy import String
from darts.metrics.metrics import r2_score 
from darts.metrics.metrics import rmse 
from plotly.subplots import make_subplots
import plotly.figure_factory as ff 
import dash_mantine_components as dmc 
from dash_iconify import DashIconify
import dash_mantine_components as dmc 
from darts.models import NBEATSModel, ExponentialSmoothing, Theta, StatsForecastAutoARIMA, AutoARIMA, RNNModel, TCNModel, BlockRNNModel, Prophet
from darts import TimeSeries 
from darts.metrics import mape, r2_score, smape 
from scipy.stats import wilcoxon, mannwhitneyu
import six  
from numpy import random as rand 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error 
import tqdm 
import plotly.io as pio 
# PATH_SOURCE_DATA laporan bulanan
df= pd.read_excel( 'D:\Fix\Data\Laporan Bulanan 2023.xlsx' ,sheet_name='2023') 
col_name = df.columns
df['Persyaratan teknis'] = df['Persyaratan teknis'].replace({'124\xa0':124,'-':0}).astype(int) 
num_cols = ['Kend. masuk','Melanggar','Tidak melanggar','Daya angkut','Dimensi','Persyaratan teknis', 
'Dokumen','Tata cara muat', 'Kelas jalan','Peringatan','Tilang','Kepolisian','Tilang uppkb lain',
'Penundaan perjalanan','Transfer muatan','Penandaan kelebihan dimensi','Penyesuaian persyaratan teknis',
'Penyesuaian tata cara muat','Putar balik','>5%-20%','21%-40%','41%-60%','61%-80%','81%-100%','>100%']
df[num_cols] = df[num_cols].replace({np.NaN:0,'-':0}).astype(int)
df['Date'] = pd.to_datetime(df['Tahun'].astype(str) + '-'+df['Bulan2'].astype(str) + '-' + '15', format="%d/%m/%Y", errors='coerce')
df.reset_index(inplace=True, drop=True)
col_dbv5 = pd.DataFrame(['Uppkb', 'Tahun', 'Bulan', 'Status pengoperasian', 'Kend. masuk','Melanggar', 'Tidak melanggar', 'Daya angkut', 'Dimensi','Persyaratan teknis', 'Dokumen', 'Tata cara muat', 'Kelas jalan','Peringatan', 'Tilang', 'Kepolisian', 'Tilang uppkb lain', 'Penundaan perjalanan', 'Transfer muatan','Penandaan kelebihan dimensi', 'Putar balik','Penyesuaian persyaratan teknis', 'Penyesuaian tata cara muat','Selisih', '% Tidak Melanggar', '% Melanggar', 'Bulan2', 'Date','Year_Month','>5%-20%','21%-40%','41%-60%','61%-80%','81%-100%','>100%'], columns=['cols'])
col_dbv4 = pd.DataFrame(df.columns, columns=['cols']) 
df['Status pengoperasian'] = df['Status pengoperasian'].str.upper()
df['% Tidak Melanggar'] = round((df['Tidak melanggar'] / df['Kend. masuk']) * 100,2)
df['% Melanggar'] = round((df['Melanggar'] / df['Kend. masuk']) * 100,2)
df['Selisih'] = df['Tidak melanggar'] - df['Melanggar']
df['Tahun'] = df['Tahun'].astype(int)
reorder_col = ['Uppkb', 'Tahun', 'Bulan', 'Status pengoperasian', 'Kend. masuk','Melanggar', 'Tidak melanggar', 'Daya angkut', 'Dimensi','Persyaratan teknis', 'Dokumen', 'Tata cara muat', 'Kelas jalan','Peringatan', 'Tilang', 'Kepolisian', 'Tilang uppkb lain','Penundaan perjalanan', 'Transfer muatan','Penandaan kelebihan dimensi', 'Putar balik','Penyesuaian persyaratan teknis', 'Penyesuaian tata cara muat','Selisih', '% Tidak Melanggar', '% Melanggar', 'Bulan2', 'Date','Year_Month','>5%-20%','21%-40%','41%-60%','61%-80%','81%-100%','>100%']
df = df[reorder_col] 
# PATH_SOURCE_DATA sdm 
path_sdm = 'D:\Fix\Data\SDM Mandiri & Pendampingan 2021 - 2023.xlsx'
sdm_pendampingan_2021 = pd.read_excel(path_sdm,sheet_name='SDM Pendampingan 2021') 
sdm_pendampingan_2021.columns = sdm_pendampingan_2021.iloc[0] 
sdm_pendampingan_2021 = sdm_pendampingan_2021.iloc[1:] 
sdm_pendampingan_2022 = pd.read_excel(path_sdm,sheet_name='SDM Pendampingan 2022')
sdm_pendampingan_2022.columns = sdm_pendampingan_2022.iloc[0] 
sdm_pendampingan_2022 = sdm_pendampingan_2022.iloc[1:]
sdm_pendampingan_2023 = pd.read_excel(path_sdm,sheet_name='SDM Pendampingan 2023') 
sdm_pendampingan_2023.columns = sdm_pendampingan_2023.iloc[0] 
sdm_pendampingan_2023 = sdm_pendampingan_2023.iloc[1:]
# PATH_SOURCE_DATA rekapitulasi capaian pengawasan
smm_header = pd.read_excel( "D:\Fix\Data\Penilaian SMM.xlsx", sheet_name='Rekap').iloc[2:3].values[0] 
bulan_data_smm = ['01','02','03','04','05','06','07'] 
filter_uppkb = list(np.sort(df['Uppkb'].unique()))
data_smm = {} 
smm_col = ['UPPKB','PRODUKTIFITAS Nasional','IMP. SOP','Konsistensi SOP','Mutu Kinerja/Penilaian Kinerja','Target 100% dalam 1 Bulan Berjalan','Jumlah Realisasi Pendataan Bulan Berjalan','Persentase Pecapaian Target Bulan Berjalan'] 
for i in bulan_data_smm: 
    # PATH_SOURCE_DATA rekapitulasi capaian pengawasan looping
    data_smm[i]=pd.read_excel('D:\Fix\Data\Penilaian SMM.xlsx', sheet_name='Rekap').iloc[3:].reset_index(drop=True)
    data_smm[i].columns = smm_header 
    data_smm[i] = data_smm[i][['UPPKB','PRODUKTIFITAS Nasional','IMP. SOP','Konsistensi SOP','Mutu Kinerja/Penilaian Kinerja','Persentase Pecapaian Target Bulan Berjalan','Target 100% dalam 1 Bulan Berjalan','Jumlah Realisasi Pendataan Bulan Berjalan']]
    k = pd.Series(data_smm[i]['UPPKB'].unique())
for i,j in data_smm.items(): 
    data_smm[i] = j.set_index('UPPKB')  
mydb = mysql.connector.connect(host = 'localhost',user = 'root', passwd = 'SadisnotOption22',database = 'jt')
mycursor = mydb.cursor() 
def sql_df(yourQuery):
    mycursor.execute(yourQuery)
    myresult = mycursor.fetchall() 
    df = pd.DataFrame(myresult, columns=mycursor.column_names)
    return df
# DATABASE_ATTRIBUTE
database_username = 'root' 
database_password = 'SadisnotOption22' 
database_ip       = '127.0.0.1'
database_name     = 'jt' 
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(database_username, database_password, database_ip, database_name)) 
home_button_alt = DashIconify(icon='material-symbols:garage-home-outline', style={'marginRight':5})
home_icon = DashIconify(icon='material-symbols:home-outline', style={"marginRight": 5})
download_icon = DashIconify(icon='material-symbols:download', style={"marginRight": 5}) 
chart_icon = DashIconify(icon='material-symbols:bar-chart-4-bars', style={"marginRight": 5}) 
forecasting_icon = DashIconify(icon='material-symbols:chart-data-outline-sharp', style={"marginRight": 5}) 
raport_icon = DashIconify(icon='material-symbols:demography-outline-sharp', style={"marginRight": 5}) 
linkedin_icon = DashIconify(icon='mdi:linkedin', style={"marginRight": 15})
button_variant = 'filled'
button_size = 'sm'
button_color = 'dark.6'
button_divider_color = 'white'
button_space = 10
SI_LOGO = 'https://www.ptsi.co.id/cfind/source/images/logo-landscape-+-ids.png'
but_group = dmc.SimpleGrid(cols=1,children=[dmc.Image(src=SI_LOGO,style={'filter':'brightness(0) invert(1)','height':'100%', 'width':'100%'}, width='100%'),dmc.Space(h=200),dmc.Anchor(dmc.Button("Home",variant=button_variant,fullWidth=True,size=button_size,leftIcon=home_icon, color=button_color),href='/home'),dmc.Space(h=button_space),dmc.Divider(label=dmc.Text('Dashboard Menu', size='md'), labelPosition="center", color=button_divider_color),dmc.Anchor(dmc.Button("Rekapitulasi",variant=button_variant,fullWidth=True,size=button_size,leftIcon=chart_icon, color=button_color), href='/'),dmc.Anchor(dmc.Button("Forecasting",variant=button_variant,fullWidth=True,size=button_size,leftIcon=forecasting_icon,color=button_color), href='/forecasting'),dmc.Anchor(dmc.Button("Raport",variant=button_variant,fullWidth=True,size=button_size,leftIcon=raport_icon,color=button_color), href='/raport'),dmc.Space(h=button_space),dmc.Divider(label=dmc.Text('Others', size='md'), labelPosition="center", color=button_divider_color),dmc.Anchor( dmc.Button("Download Data", variant=button_variant,gradient={"from": "lime", "to": "cyan"}, fullWidth=True,size=button_size,leftIcon=download_icon,color=button_color),href='/download_data'),dmc.Space(h=button_space),dmc.Divider(label=dmc.Text('Contact Us', size='md'), labelPosition="center", color=button_divider_color),dmc.Anchor(dmc.Button("Linkedin",variant=button_variant,gradient={"from": "lime", "to": "cyan"},fullWidth=True,size=button_size,leftIcon=linkedin_icon,color=button_color),href='linked_in')],spacing=10)
lay_sidebar = dbc.Container([html.Br(),but_group],class_name='shadow p-3 mb-5',style={'height':'100%',"position": "fixed","top": 0,"left": 0, "bottom": 0,"width": "14rem","padding": "2rem 1rem",'background-color': '#101113'})
yearrr = list(df['Tahun'].unique()) 
lay_filter = dbc.CardGroup([dbc.Card([dmc.Text('Komparasi (Pendampingan vs Mandiri):',weight=200,align='center',style={"fontSize": 15}),dmc.Center(dbc.Checklist(options=[ {"value": 1},],value=[1],id="filter_komparasi",switch=True))],class_name='bg-transparent border-end p-3'),dbc.Card([dbc.Col([dbc.Row(html.Span('Pilih Tahun :')),dbc.Row([dcc.RangeSlider(id='filter_tahun',min=int(df['Tahun'].min()),max=int(df['Tahun'].max()),step=1,marks=None,tooltip={"placement": "bottom", "always_visible": True},value= yearrr,allowCross=True)])])], class_name='bg-transparent border-start p-3')])
filter_icon = DashIconify(icon="mdi:filter", style={"marginTop":0, 'font-size':'15px', 'padding':'0px'})
coba_filter = dmc.Menu([dmc.MenuTarget( dmc.Button('Filter', leftIcon=filter_icon, radius='sm', size='sm', variant='filled', color='dark.9', compact=True)),dmc.MenuDropdown([lay_filter]),])
univariate_cat = dbc.Card([dbc.Row([dbc.Label('Pilih Variable :'),html.Br(),dcc.Dropdown( id='univariate_dropdown_categoric',options=['Jenis Pelanggaran', 'Jenis Penindakan', 'Jenis Penanganan'],value='Jenis Pelanggaran') ]),dbc.Row([dcc.Graph(id='univariate_figure_categoric')]),],class_name='shadow p-3 mb-2 bg-white rounded border-0',style={'margin':'5px'}) 
type_graph = ['boxplot','histogram','violinplot', 'descriptive statistics']
univ_var_num = ['Kend. masuk','Melanggar','Tidak melanggar','Daya angkut','Dimensi','Persyaratan teknis','Dokumen','Tata cara muat','Kelas jalan','Peringatan','Tilang','Kepolisian','Tilang uppkb lain','Penundaan perjalanan','Transfer muatan','Penandaan kelebihan dimensi','Putar balik','Penyesuaian persyaratan teknis','Penyesuaian tata cara muat']
univariate_num = dbc.Card(dbc.Row([dbc.Row([dbc.Col( [html.Span('Pilih Variable :'),html.Br(),dcc.Dropdown(id='univariate_dropdown_numeric_var',options=univ_var_num,value='Kend. masuk')]),dbc.Col([html.Span('Pilih Visualisasi :'),dcc.Dropdown( id='univariate_dropdown_numeric_graph_type',options=type_graph,value='boxplot')])]),dbc.Row([dcc.Graph(id='univariate_figure_numeric')])]) ,class_name='shadow p-3 mb-2 bg-white rounded border-0',style={'margin':'5px'}) 
combined_univariate = dbc.CardGroup([univariate_cat, univariate_num  ]) 
title_bivariate = dbc.Row([dbc.Col([html.H4('Bivariate Analysis',className='text-center text-black-50')])])
corr_var = ['Kend. masuk','Melanggar','Tidak melanggar','Daya angkut','Dimensi','Persyaratan teknis', 'Dokumen','Tata cara muat','Kelas jalan','Peringatan','Tilang','Kepolisian','Tilang uppkb lain','Penundaan perjalanan','Transfer muatan','Penandaan kelebihan dimensi','Putar balik','Penyesuaian persyaratan teknis','Penyesuaian tata cara muat']
corr_var2 = corr_var.copy() 
correlation = dbc.Card(dbc.Row([dbc.Row([dbc.Col([html.Span('Pilih Variable X :'),dcc.Dropdown(id='corr_option_x', options=corr_var,value='Kend. masuk')]), dbc.Col([html.Span('Pilih Variable Y :'),dcc.Dropdown(id='corr_option_y', options=corr_var2,value='Melanggar')])]),dbc.Row([dcc.Graph(id='corr_graph')])]),class_name='shadow p-3 mb-2 bg-white rounded border-0')
title_3d_plot = dbc.Row([dbc.Col([html.H4('Multivariate Analysis',className='text-center text-black-50')])])
_3d_var = ['Uppkb','Tahun','Bulan','Status pengoperasian','Kend. masuk','Melanggar','Tidak melanggar','Daya angkut','Dimensi','Persyaratan teknis','Dokumen','Tata cara muat','Kelas jalan','Peringatan','Tilang','Kepolisian','Tilang uppkb lain','Penundaan perjalanan','Transfer muatan','Penandaan kelebihan dimensi','Putar balik','Penyesuaian persyaratan teknis','Penyesuaian tata cara muat']
plot_3d = dbc.Card(dbc.Row( [dbc.Row([dbc.Col([html.Span('Pilih Variable X :'),html.Br(),dcc.Dropdown(id='3d_option_x', options=_3d_var,value='Status pengoperasian')]), dbc.Col([html.Span('Pilih Variable Y :'),html.Br(),    dcc.Dropdown(id='3d_option_y', options=_3d_var,value='Kend. masuk')]),dbc.Col([html.Span('Pilih Variable X :'),html.Br(),dcc.Dropdown(id='3d_option_z', options=_3d_var,value='Melanggar')])]), dbc.Row([ dcc.Graph(id='3d_graph')])]),class_name='shadow p-3 mb-5 bg-white rounded border-0') 
color_map = {'MANDIRI':'#FF2E63','PENDAMPINGAN':'#00ADB5'}
color_map_all_uppkb = {'ALL UPPKB':'#11009E'} 
title_indicator = dmc.Center([dmc.Text('Indicator',weight=500,align='left',style={"fontSize": 25})],style={'width':'100%'}) 
indicator_component =  dbc.Card(dbc.Row([dbc.Row([dbc.Col([html.Span('Pilih Variable :'),html.Br(),dcc.Dropdown(id='line_ind_var',options=univ_var_num,value='Kend. masuk')])]),dbc.Row([dbc.Col([dcc.Graph(id='line_indicator_figure')])])]),class_name='shadow p-3 mb-2 bg-white rounded border-0',style={'margin':'5px'})
line_component = dbc.Card([dbc.Row([dbc.Col([dcc.Graph(id='line_figure')])])],class_name='shadow p-3 mb-2 bg-white rounded border-0',style={'margin':'5px'})
combined_line = dmc.Grid(children=[dmc.Col([indicator_component], span=3),dmc.Col(line_component, span=9)],gutter="sm") 
burger_icon = DashIconify(icon='mdi:menu' , style={"marginTop":30, 'font-size':'35px', 'padding':'0px'})
burger_button = dmc.ActionIcon(burger_icon,id="open-offcanvas", n_clicks=0, radius='xs', size='xl',variant='transparent')
offcanvas = html.Div([burger_button,dbc.Offcanvas(lay_sidebar,id="offcanvas",is_open=False,scrollable=True,backdrop=True,close_button=True,style={'width': '14rem'}),])
lay_main_title_rekapitulasi=dmc.Grid(children=[dmc.Col([dmc.SimpleGrid([dmc.Text('Rekapitulasi',weight=700,align='left',style={"fontSize": 50}, color='dark.9'),], cols=2, spacing=0)],span=1),dmc.Col([coba_filter], offset=9 ,span=2)],gutter='xs')
title_font_size = 25
title_font_color = 'dark.5'
title_indicator = dmc.Center([dmc.HoverCard(withArrow=True,width=300,shadow="lg",children=[dmc.HoverCardTarget(dmc.Text('Indicator',weight=500,style={"fontSize": title_font_size}, color=title_font_color)),dmc.HoverCardDropdown(dmc.Text("Indicator digunakan untuk menggambarkan trend kondisi variable secara singkat",size="md",)),],)],style={'width':'100%'})
title_univariate = dmc.Center([dmc.HoverCard(withArrow=True,width=300,shadow="lg",children=[dmc.HoverCardTarget(dmc.Text('Univariate Analysis',weight=500,style={"fontSize": title_font_size}, color=title_font_color)),dmc.HoverCardDropdown(dmc.Text("Univariate Analysis adalah suatu teknik analisis data terhadap satu variabel secara mandiri, tiap variabel dianalisis tanpa dikaitkan dengan variabel lainnya",size="md",)),],)],style={'width':'100%'}) 
combined_sub_title_univariate = dbc.CardGroup([dbc.Card([dmc.Center([dmc.Text('Categoric',weight=500,style={"fontSize": 20},color=title_font_color)],style={'width':'100%'})],class_name='bg-transparent border-0 p-0'),dbc.Card([dmc.Center([dmc.Text('Numeric',weight=500,style={"fontSize": 20},color=title_font_color)],style={'width':'100%'})],class_name='bg-transparent border-0 p-0')])
title_bivariate = dmc.Center([dmc.HoverCard( withArrow=True,width=300,shadow="lg",children=[dmc.HoverCardTarget(dmc.Text('Bivariate Analysis',weight=500,style={"fontSize": title_font_size}, color=title_font_color)),dmc.HoverCardDropdown(dmc.Text('Bivariate Analysis merupakan analisis yang dilakukan untuk mengetahui hubungan antara 2 variabel.',size="md",)),],)],style={'width':'100%'})
title_multivariate = dmc.Center([dmc.HoverCard( withArrow=True,width=300,shadow="lg",children=[dmc.HoverCardTarget(dmc.Text('Multivariate Analysis',weight=500,style={"fontSize": title_font_size}, color=title_font_color)),dmc.HoverCardDropdown(dmc.Text('Multivariate Analysis adalah metode pengolahan variabel dalam jumlah yang banyak, dimana tujuannya adalah untuk mencari pengaruh variabel-variabel tersebut terhadap suatu obyek secara simultan atau serentak',size="md",)),],)],style={'width':'100%'})
title_stat_test = dmc.Center([dmc.HoverCard(withArrow=True,width=300,shadow="lg",children=[dmc.HoverCardTarget(dmc.Text('Statistical Test',weight=500,style={"fontSize": title_font_size}, color=title_font_color)),dmc.HoverCardDropdown(dmc.Text("Statistical Test adalah teknik analisis statistik yang digunakan untuk menguji hipotesis, membandingkan dua atau lebih sampel, serta menentukan hubungan antara dua variabel atau lebih.",size="md",)),],)],style={'width':'100%'}) 
lay_main_title_forecasting=dmc.Grid(children=[dmc.Col([dmc.SimpleGrid([dmc.Text('Predictive Analytics',weight=700,align='left',style={"fontSize": 50}, color='dark.9'),], cols=2, spacing=0)],span=12),],gutter='xs')
title_forecasting_melanggar = dmc.Center([dmc.HoverCard(withArrow=True,width=300,shadow="lg",children=[dmc.HoverCardTarget(dmc.Text('Forecasting Persentase Kendaraan Melanggar',weight=500,style={"fontSize": title_font_size}, color=title_font_color)),dmc.HoverCardDropdown(dmc.Text("Forecasting ini digunakan untuk menggambarkan trend kondisi persentase kendaraan melanggar dimasa depan",size="md",)),],)],style={'width':'100%'})
title_komoditi = dmc.Center([dmc.HoverCard(withArrow=True,width=300,shadow="lg",children=[dmc.HoverCardTarget( dmc.Text('Trend dan Persebaran Komoditas',weight=500,style={"fontSize": title_font_size}, color=title_font_color)),dmc.HoverCardDropdown(dmc.Text("Map ini digunakan untuk menggambarkan trend dan persebaran komoditas",size="md",)),],)],style={'width':'100%'})
sub_title_forecasting = dbc.CardGroup([dbc.Card([dmc.Center([dmc.Text('Model Evaluation',weight=500,style={"fontSize": 20},color=title_font_color)],style={'width':'100%'})],class_name='bg-transparent border-0 p-0'),dbc.Card([dmc.Center([dmc.Text('Forecasting Data',weight=500,style={"fontSize": 20},color=title_font_color)],style={'width':'100%'})],class_name='bg-transparent border-0 p-0')])
# PATH_SOURCE_DATA sdm
path_sdm = 'D:\Fix\Data\SDM Mandiri & Pendampingan 2021 - 2023.xlsx'
sdm_mandiri_2021 = pd.read_excel(path_sdm,sheet_name='SDM Mandiri 2021')
sdm_mandiri_2021.columns = sdm_mandiri_2021.iloc[0]
sdm_mandiri_2021=sdm_mandiri_2021.iloc[1:] 
sdm_mandiri_2022 = pd.read_excel(path_sdm,sheet_name='SDM Mandiri 2022')
sdm_mandiri_2023 = pd.read_excel(path_sdm,sheet_name='SDM Mandiri 2023')
sdm_pendampingan_2021 = pd.read_excel(path_sdm,sheet_name='SDM Pendampingan 2021') 
sdm_pendampingan_2021.columns = sdm_pendampingan_2021.iloc[0] 
sdm_pendampingan_2021 = sdm_pendampingan_2021.iloc[1:] 
sdm_pendampingan_2022 = pd.read_excel(path_sdm,sheet_name='SDM Pendampingan 2022')
sdm_pendampingan_2022.columns = sdm_pendampingan_2022.iloc[0] 
sdm_pendampingan_2022 = sdm_pendampingan_2022.iloc[1:]
sdm_pendampingan_2023 = pd.read_excel(path_sdm,sheet_name='SDM Pendampingan 2023') 
sdm_pendampingan_2023.columns = sdm_pendampingan_2023.iloc[0] 
sdm_pendampingan_2023 = sdm_pendampingan_2023.iloc[1:] 
map_b = {'Januari':'January','Februari':'February','Maret':'March','Mei':'May', 'Juni':'June', 'Juli':'July','Agustus':'August','Oktober':'October','Desember':'December'} 
map_bulan = {'January':'1','February':'2','March':'3','April':'4','May':'5','June':'6','July':'7','August':'8','September':'9','October':'10','November':'11','December':'12'} 
def create_data_pendampingan(tahun): 
    df = sql_df("""
    WITH 
        Melanggar AS ( 
            SELECT Bulan, count(*) as 'Melanggar' FROM jt.master{th}
            WHERE
                (`Status Melanggar/Tidak` = 'Melanggar') & 
                (`Nama JT` IN ('CIBARAGALAN', 'DATAE', 'DOLOK ESTATE LIMA PULUH',
       'INOBONTO', 'KALIBARU MANIS', 'KAYU MALUE', 'KINTAP', 'LUBUK SELASIH', 'MERAPI',
       'MUARA LEMBU', 'MUARA TAMBESI', 'POJOK', 'SARANG', 'SUBAH','SUNGAI LANGSAT', 'TAMAN MARTANI','TANA BATUE','TANJUK BALIK','TOMO','WATUDODOL','WIDANG'))
            group by Bulan), 
        Tidak_Melanggar AS (
            SELECT Bulan, count(*) as 'Tidak_Melanggar' FROM jt.master{th}
            WHERE (`Status Melanggar/Tidak` = 'Tidak Melanggar') &
                (`Nama JT` IN ('CIBARAGALAN', 'DATAE', 'DOLOK ESTATE LIMA PULUH',
       'INOBONTO', 'KALIBARU MANIS', 'KAYU MALUE', 'KINTAP', 'LUBUK SELASIH', 'MERAPI',
       'MUARA LEMBU', 'MUARA TAMBESI', 'POJOK', 'SARANG', 'SUBAH','SUNGAI LANGSAT', 'TAMAN MARTANI','TANA BATUE','TANJUK BALIK','TOMO','WATUDODOL','WIDANG')) 
            group by Bulan),
        Kend_masuk AS ( 
        SELECT Bulan, count(*) as 'Kend_Masuk' FROM jt.master{th}
        WHERE `Nama JT` IN ('CIBARAGALAN', 'DATAE', 'DOLOK ESTATE LIMA PULUH',
       'INOBONTO', 'KALIBARU MANIS', 'KAYU MALUE', 'KINTAP', 'LUBUK SELASIH', 'MERAPI',
       'MUARA LEMBU', 'MUARA TAMBESI', 'POJOK', 'SARANG', 'SUBAH','SUNGAI LANGSAT', 'TAMAN MARTANI','TANA BATUE','TANJUK BALIK','TOMO','WATUDODOL','WIDANG')
        GROUP BY Bulan)
    SELECT k.Bulan, k.Kend_Masuk, m.Melanggar, tm.Tidak_Melanggar FROM Kend_masuk AS k
    LEFT JOIN Melanggar AS m ON m.Bulan = k.Bulan
    LEFT JOIN Tidak_Melanggar AS tm ON m.Bulan = tm.Bulan;
    """.format(th=tahun))
    df = df.replace({np.NaN:0}) 
    df['Bulan'] = df['Bulan'].replace(map_b).replace(map_bulan).astype(int)
    df.insert(0,'Tahun',tahun) 
    c = pd.Series(range(1,13), name='com_num').astype(int)
    c[~c.isin(df['Bulan'] )].T 
    df_imput = pd.DataFrame({'Bulan':c[~c.isin(df['Bulan'] )].T }).astype(int)
    df_imput.insert(0,'Tahun',tahun) 
    df_imput.insert(2,'Kend_Masuk',0)
    df_imput.insert(3,'Melanggar',0)
    df_imput.insert(4,'Tidak_Melanggar',0)  
    df_final = pd.concat([df_imput,df]).sort_values(by='Bulan', ignore_index=True)
    df_final['Y_M'] = '{th}'.format(th=tahun) + '-' + df_final['Bulan'].astype(str)
    df_final['%Melanggar'] = round((df_final['Melanggar'] / df_final['Kend_Masuk']) * 100)
    df_final['%Tidak Melanggar'] = round((df_final['Tidak_Melanggar'] / df_final['Kend_Masuk']) * 100)
    df_final = df_final.replace({np.NaN:0})
    return df_final 
def create_data_mandiri(tahun): 
    df = sql_df("""
    WITH
        Melanggar AS (
            SELECT Bulan, count(*) as 'Melanggar' FROM jt.master{th} 
            WHERE 
                (`Status Melanggar/Tidak` = 'Melanggar') & 
                (`Nama JT` NOT IN ('CIBARAGALAN', 'DATAE', 'DOLOK ESTATE LIMA PULUH',
       'INOBONTO', 'KALIBARU MANIS', 'KAYU MALUE', 'KINTAP', 'LUBUK SELASIH', 'MERAPI',
       'MUARA LEMBU', 'MUARA TAMBESI', 'POJOK', 'SARANG', 'SUBAH','SUNGAI LANGSAT', 'TAMAN MARTANI','TANA BATUE','TANJUK BALIK','TOMO','WATUDODOL','WIDANG'))
            group by Bulan), 
        Tidak_Melanggar AS (
            SELECT Bulan, count(*) as 'Tidak_Melanggar' FROM jt.master{th}
            WHERE (`Status Melanggar/Tidak` = 'Tidak Melanggar') &
                (`Nama JT` NOT IN ('CIBARAGALAN', 'DATAE', 'DOLOK ESTATE LIMA PULUH',
       'INOBONTO', 'KALIBARU MANIS', 'KAYU MALUE', 'KINTAP', 'LUBUK SELASIH', 'MERAPI',
       'MUARA LEMBU', 'MUARA TAMBESI', 'POJOK', 'SARANG', 'SUBAH','SUNGAI LANGSAT', 'TAMAN MARTANI','TANA BATUE','TANJUK BALIK','TOMO','WATUDODOL','WIDANG')) 
            group by Bulan),
        Kend_masuk AS ( 
        SELECT Bulan, count(*) as 'Kend_Masuk' FROM jt.master{th}
        WHERE `Nama JT` NOT IN ('CIBARAGALAN', 'DATAE', 'DOLOK ESTATE LIMA PULUH',
       'INOBONTO', 'KALIBARU MANIS', 'KAYU MALUE', 'KINTAP', 'LUBUK SELASIH', 'MERAPI',
       'MUARA LEMBU', 'MUARA TAMBESI', 'POJOK', 'SARANG', 'SUBAH','SUNGAI LANGSAT', 'TAMAN MARTANI','TANA BATUE','TANJUK BALIK','TOMO','WATUDODOL','WIDANG')
        GROUP BY Bulan) 
    SELECT k.Bulan, k.Kend_Masuk, m.Melanggar, tm.Tidak_Melanggar FROM Kend_masuk AS k
    LEFT JOIN Melanggar AS m ON m.Bulan = k.Bulan
    LEFT JOIN Tidak_Melanggar AS tm ON m.Bulan = tm.Bulan;
    """.format(th=tahun)) 
    df = df.replace({np.NaN:0})  
    df['Bulan'] = df['Bulan'].replace(map_b).replace(map_bulan).astype(int)
    df.insert(0,'Tahun',tahun) 
    c = pd.Series(range(1,13), name='com_num').astype(int)
    c[~c.isin(df['Bulan'] )].T 
    df_imput = pd.DataFrame({'Bulan':c[~c.isin(df['Bulan'] )].T }).astype(int)
    df_imput.insert(0,'Tahun',tahun) 
    df_imput.insert(2,'Kend_Masuk',0)
    df_imput.insert(3,'Melanggar',0)
    df_imput.insert(4,'Tidak_Melanggar',0)
    df_final = pd.concat([df_imput,df]).sort_values(by='Bulan', ignore_index=True)
    df_final['Y_M'] = '{th}'.format(th=tahun) + '-' + df_final['Bulan'].astype(str)
    df_final['%Melanggar'] = round((df_final['Melanggar'] / df_final['Kend_Masuk']) * 100)
    df_final['%Tidak Melanggar'] = round((df_final['Tidak_Melanggar'] / df_final['Kend_Masuk']) * 100)
    df_final = df_final.replace({np.NaN:0})
    return df_final
def create_data_all(tahun):
    df = sql_df("""
    WITH
        Melanggar AS ( 
            SELECT Bulan, count(*) as 'Melanggar' FROM jt.master{th} 
            WHERE 
                (`Status Melanggar/Tidak` = 'Melanggar')
            group by Bulan), 
        Tidak_Melanggar AS ( 
            SELECT Bulan, count(*) as 'Tidak_Melanggar' FROM jt.master{th}
            WHERE (`Status Melanggar/Tidak` = 'Tidak Melanggar')
            group by Bulan), 
        Kend_masuk AS (
        SELECT Bulan, count(*) as 'Kend_Masuk' FROM jt.master{th}
        GROUP BY Bulan)
    SELECT k.Bulan, k.Kend_Masuk, m.Melanggar, tm.Tidak_Melanggar FROM Kend_masuk AS k
    LEFT JOIN Melanggar AS m ON m.Bulan = k.Bulan
    LEFT JOIN Tidak_Melanggar AS tm ON m.Bulan = tm.Bulan;
    """.format(th=tahun)) 
    df['Bulan'] = df['Bulan'].replace(map_b).replace(map_bulan)
    df['Y_M'] = '{th}'.format(th=tahun) + '-' + df['Bulan'].astype(str)
    df.insert(0,column='Tahun',value=[str.split(i,'-')[0] for i in df['Y_M']]) 
    df['%Melanggar'] = round((df['Melanggar'] / df['Kend_Masuk']) * 100)
    df['%Tidak Melanggar'] = round((df['Tidak_Melanggar'] / df['Kend_Masuk']) * 100)
    df = df.replace({np.NaN:0}) 
    return df 
def all_data_yearly():
    table_all = []
    table_pendampingan = []
    table_mandiri = []
    for i in df['Tahun'].unique():
        table_all.append(create_data_all(i))
        table_pendampingan.append(create_data_pendampingan(i))
        table_mandiri.append(create_data_mandiri(i))
    return dict(all = table_all,pendampingan = table_pendampingan,mandiri = table_mandiri)
def stat_test_new(df_before, df_after): 
    table = []
    for i in ['Kend_Masuk', 'Melanggar', 'Tidak_Melanggar']:
        y = []
        alpha = 0.05 
        df_paired = pd.DataFrame({'{th}'.format(th=df_before['Tahun'].unique()[0]):df_before[i], '{th}'.format(th=df_after['Tahun'].unique()[0]):df_after[i]})
        stat, p_value = wilcoxon(df_before[i],df_after[i], alternative= 'two-sided')
        if p_value > alpha:
            y.append('Same distribution (fail to reject H0)')
        else: 
            y.append('Different distribution (reject H0)')
        table.append(pd.DataFrame({'Median {th}'.format(th=df_before['Tahun'].unique()[0]):round(df_before[i].median(),2),'Median {th}'.format(th=df_after['Tahun'].unique()[0]):round(df_after[i].median(),2),'Alpha': alpha,'Statistics': '%.3f, p=%.3f' % (stat, p_value),'Conclusion': y}).transpose().rename(columns={0:'Wilcoxon Statistical Test'}).reset_index().rename(columns={'index':'Attribute'}))
    return dict(Kend_Masuk=table[0],Melanggar=table[1],Tidak_Melanggar=table[2])
coba = all_data_yearly() 
result_all = [] 
for i in range(1,len(coba['all'])-1):
    result_all.append(stat_test_new(coba['all'][i-1], coba['all'][i]))
result_pendampingan = []
for i in range(1,len(coba['pendampingan'])):
    result_pendampingan.append(stat_test_new(coba['pendampingan'][i-1], coba['pendampingan'][i]))
result_mandiri = [] 
for i in range(1,len(coba['mandiri'])): 
    result_mandiri.append(stat_test_new(coba['mandiri'][i-1], coba['mandiri'][i]))
coba_pendampingan = coba['pendampingan']
coba_mandiri = coba['mandiri'] 
coba_all = coba['all']  
t = [] 
for i in coba['all']: 
    t.append(int(i['Tahun'].unique()[0]))
opt_tahun_wilcoxon  = t[1:]
opt_tahun_map_wilcoxon = {}
for i,j in zip(opt_tahun_wilcoxon, range(len(opt_tahun_wilcoxon))):
    opt_tahun_map_wilcoxon[i] = j
opt_wilcoxon = ['Kend_Masuk','Melanggar','Tidak_Melanggar']
opt_map_wilcoxon = {} 
for i, j in zip(opt_wilcoxon,['Kendaraan Masuk', 'Melanggar', 'Tidak Melanggar']):
    opt_map_wilcoxon[i] = j
table_wilcoxon = dbc.Card([dbc.Row([dbc.Col([html.Span('Pilih Variable :'),html.Br(),dcc.Dropdown(id='var_wilcoxon',options=opt_wilcoxon,value='Kend_Masuk')]),dbc.Col([html.Span('Pilih Tahun:'),html.Br(),dcc.Dropdown(id='tahun_wilcoxon',options=opt_tahun_wilcoxon,value=2022)]),dbc.Col([html.Span('Pilih Visualisasi:'),html.Br(),dcc.Dropdown(id='viz_wilcoxon',options=['Graph', 'Statistical Test'],value='Graph')])]),dbc.Row([dcc.Graph(id='table_wilcoxon')])],class_name='shadow p-3 mb-2 bg-white rounded border-0')
def data_monthly_all(tahun):
    df = sql_df(""" 
    WITH 
        Melanggar AS ( -- Melanggar 
            SELECT {th} as 'Tahun', MONTH(Tanggal) month_number,count(*) as 'Melanggar' FROM jt.master{th}
            WHERE (`Status Melanggar/Tidak` = 'Melanggar')
            group by month_number),
        Tidak_Melanggar AS ( -- Tidak_Melanggar 
            SELECT {th} as 'Tahun', MONTH(Tanggal) month_number,count(*) as 'Tidak_Melanggar' FROM jt.master{th}
            WHERE (`Status Melanggar/Tidak` = 'Tidak Melanggar')
            GROUP BY month_number),
        Kend_masuk AS ( -- Kend_masuk
            SELECT {th} as 'Tahun', MONTH(Tanggal) month_number, count(*) as 'Kend_Masuk' FROM jt.master{th}
            GROUP BY month_number) 
    SELECT k.month_number, k.Kend_Masuk, m.Melanggar, tm.Tidak_Melanggar FROM Kend_masuk AS k
    LEFT JOIN Melanggar AS m ON m.month_number = k.month_number
    LEFT JOIN Tidak_Melanggar AS tm ON k.month_number = tm.month_number;
    """.format(th=tahun)) 
    df.insert(0, 'Tahun', tahun) 
    df = df.replace({np.NaN:0})
    c = pd.Series(range(1,13), name='com_num') 
    df_imput = pd.DataFrame({'month_number':c[~c.isin(df['month_number'] )].T }) 
    df_imput.insert(0,'Tahun',tahun)
    df_imput.insert(2,'Kend_Masuk',0) 
    df_imput.insert(3,'Melanggar',0)
    df_imput.insert(4,'Tidak_Melanggar',0) 
    df_final = pd.concat([df_imput,df]).sort_values(by='month_number', ignore_index=True)
    df_final = df_final.astype(int)
    df_final['%Melanggar'] = round((df_final['Melanggar'] / df_final['Kend_Masuk']) * 100,2)
    df_final['%Tidak Melanggar'] = round((df_final['Tidak_Melanggar'] / df_final['Kend_Masuk']) * 100,2)
    df_final = df_final.replace({np.NaN:0}) 
    df_final.rename(columns={'month_number':'Bulan'}, inplace=True)
    return df_final
data_monthly_nasional = pd.concat([data_monthly_all(2020),data_monthly_all(2021),data_monthly_all(2022),data_monthly_all(2023).iloc[:-1]], ignore_index=True)
def data_weekly_all(tahun):
    df = sql_df(""" 
    WITH
        Melanggar AS ( -- Melanggar 
            SELECT WEEK(Tanggal,1) weeknumber,count(*) as 'Melanggar' FROM jt.master{th}
            WHERE (`Status Melanggar/Tidak` = 'Melanggar')
            group by weeknumber),
        Tidak_Melanggar AS ( -- Tidak_Melanggar 
            SELECT WEEK(Tanggal,1) weeknumber,count(*) as 'Tidak_Melanggar' FROM jt.master{th}
            WHERE (`Status Melanggar/Tidak` = 'Tidak Melanggar')
            GROUP BY weeknumber),
        Kend_masuk AS ( -- Kend_masuk
            SELECT WEEK(Tanggal,1) weeknumber, count(*) as 'Kend_Masuk' FROM jt.master{th}
            GROUP BY weeknumber) 
    SELECT k.weeknumber, k.Kend_Masuk, m.Melanggar, tm.Tidak_Melanggar FROM Kend_masuk AS k
    LEFT JOIN Melanggar AS m ON m.weeknumber = k.weeknumber
    LEFT JOIN Tidak_Melanggar AS tm ON k.weeknumber = tm.weeknumber;
    """.format(th=tahun)) 
    df.insert(0, 'Tahun', tahun) 
    df = df.replace({np.NaN:0}) 
    c = pd.Series(range(1,54), name='com_num')
    df_imput = pd.DataFrame({'weeknumber':c[~c.isin(df['weeknumber'] )].T }) 
    df_imput.insert(0,'Tahun',tahun)
    df_imput.insert(2,'Kend_Masuk',0) 
    df_imput.insert(3,'Melanggar',0)
    df_imput.insert(4,'Tidak_Melanggar',0) 
    df_final = pd.concat([df_imput,df]).sort_values(by='weeknumber', ignore_index=True)
    df_final = df_final.astype(int) 
    df_final['%Melanggar'] = round((df_final['Melanggar'] / df_final['Kend_Masuk']) * 100,2)
    df_final['%Tidak Melanggar'] = round((df_final['Tidak_Melanggar'] / df_final['Kend_Masuk']) * 100,2)
    df_final = df_final.replace({np.NaN:0}) 
    return df_final 
y20 = data_weekly_all(2020)[1:]
y21 = data_weekly_all(2021)
y22 = data_weekly_all(2022).iloc[:53,:]
y23 = data_weekly_all(2023).iloc[:49,:]
y21.iloc[1,2:3] = pd.DataFrame(y21.iloc[1] + y21.iloc[0]).T['Kend_Masuk']
y21.iloc[1,3:4] = pd.DataFrame(y21.iloc[1] + y21.iloc[0]).T['Melanggar']
y21.iloc[1,4:5] = pd.DataFrame(y21.iloc[1] + y21.iloc[0]).T['Tidak_Melanggar'] 
y21.iloc[1,6:7] = round((y21.iloc[1]['Melanggar'] / y21.iloc[1]['Kend_Masuk']) * 100,2)
y21.iloc[1,7:8] = round((y21.iloc[1]['Tidak_Melanggar'] / y21.iloc[1]['Kend_Masuk']) * 100,2) 
y21 = y21.iloc[1:-1]
y22.iloc[1,2:3] = pd.DataFrame(y22.iloc[1] + y22.iloc[0]).T['Kend_Masuk']
y22.iloc[1,3:4] = pd.DataFrame(y22.iloc[1] + y22.iloc[0]).T['Melanggar']
y22.iloc[1,4:5] = pd.DataFrame(y22.iloc[1] + y22.iloc[0]).T['Tidak_Melanggar'] 
y22.iloc[1,6:7] = round((y22.iloc[1]['Melanggar'] / y22.iloc[1]['Kend_Masuk']) * 100,2)
y22.iloc[1,7:8] = round((y22.iloc[1]['Tidak_Melanggar'] / y22.iloc[1]['Kend_Masuk']) * 100,2) 
y22 = y22.iloc[1:] 
data_weekly_nasional = pd.concat([y20,y21,y22,y23],ignore_index=True) 
model = StatsForecastAutoARIMA() 
series = TimeSeries.from_dataframe(data_weekly_nasional,value_cols='%Melanggar') 
training, validation = series.split_after(0.8)
model.fit(training)
pred_val=model.predict(n=len(validation),num_samples=500) 
model.fit(series )
prediksi_6_bulan = 26
pred=model.predict(n=prediksi_6_bulan, num_samples=500)
rmse = round(np.sqrt(mean_squared_error(y_true=validation.pd_series(), y_pred=pred_val.quantile_df(0.5)[:len(validation.pd_series())])),2)
mae = round(mean_absolute_error(y_true=validation.pd_series(), y_pred=pred_val.quantile_df(0.5)[:len(validation.pd_series())]), ndigits=2)
mape = round(mean_absolute_percentage_error(y_true=validation.pd_series(), y_pred=pred_val.quantile_df(0.5)[:len(validation.pd_series())])*100, ndigits=2) 
pred_val_mean = round(pred_val.quantile_df(0.5)['%Melanggar_0.5'].mean(), ndigits=2)
val_mean = round(validation.pd_series().mean(), ndigits=2)
pred_mean = round(pred.quantile_df(0.5)['%Melanggar_0.5'].mean() , ndigits=2)
diff = round(abs(pred_val.quantile_df(0.5)['%Melanggar_0.5'].mean()-validation.pd_series().mean()), ndigits=2)
score = pd.DataFrame({'Attribute':['RMSE','MAE','MAPE','Validation Mean', 'Prediction Validation Mean', 'Differences','Prediction Mean'], 'Value':[rmse,mae,mape,val_mean, pred_val_mean,diff, pred_mean]}) 
len_index= len(pd.concat([pd.Series(training.pd_dataframe().index),pd.Series(validation.pd_dataframe().index),pd.Series(pred.quantile_df(0.5)['%Melanggar_0.5'].index)],ignore_index=True))
len_nticks= len(data_monthly_nasional)
n_ticks = data_monthly_nasional.iloc[:len_nticks].shape[0]
tickvals = np.linspace(0,len_index, num=n_ticks, endpoint=True)  
train_ticks = []
for i in range(1,13):
    for j in range(data_weekly_nasional['Tahun'].min(),data_weekly_nasional['Tahun'].max()+1):
        train_ticks.append(pd.to_datetime("{j}-{i}".format(j=j,i=i)))
train_ticks = pd.Series(train_ticks).sort_values(ignore_index=True).dt.strftime('%Y-%m').iloc[:n_ticks]
pred_ticks = [] 
tahun_terakhir_train = int(train_ticks.iloc[-1][0:4])
bulan_terakhir_train =int(train_ticks.iloc[-1][5:7]) 
if (data_weekly_nasional['Tahun'].max() == tahun_terakhir_train ) & (bulan_terakhir_train == 12):
    for i in range(1,7): 
        pred_ticks.append(pd.to_datetime("{j}-{i}".format(j=data_weekly_nasional['Tahun'].max()+1,i=i)))
    pred_ticks = pd.Series(pred_ticks).sort_values(ignore_index=True).dt.strftime('%Y-%m')
else: 
    for i in range(bulan_terakhir_train+1,13):
        pred_ticks.append(pd.to_datetime("{j}-{i}".format(j=data_weekly_nasional['Tahun'].max(),i=i)))
    pred_ticks = pd.Series(pred_ticks).sort_values(ignore_index=True).dt.strftime('%Y-%m')
ticktext = pd.concat([train_ticks,pred_ticks], ignore_index=True) 
data_melanggar_nasional_pred = pd.Series([ round(i.mean(),2) for i in np.split(pred.quantile_df(0.5)['%Melanggar_0.5'][:-2] ,6)])
tabel_forecasting_ = pd.concat([ticktext,pd.concat([data_monthly_nasional['%Melanggar'].iloc[:len_nticks],data_melanggar_nasional_pred],ignore_index=True)],axis=1).rename(columns={0:'Date', 1:'% Melanggar'})
tabel_forecasting_['color'] = '#C9EEFF'
tabel_forecasting_.iloc[:-7:-1, 2] = '#FDCEDF'  
option_var_forecast = ['Kendaraan Masuk', '%Melanggar']
option_stat_forecast = ['Nasional', 'Pendampingan', 'Mandiri']
Forecasting_component = dbc.Card(dbc.Row([dbc.Row([dcc.Graph(id='forecasting_graph')]),dmc.Divider(size="sm",color="#ADB5BD"),dbc.Row([sub_title_forecasting]),dbc.Row([dbc.Col([dcc.Graph(id='score_graph')]),dbc.Col([dcc.Graph(id='tabel_forecasting_graph')])])]),class_name='shadow p-3 mb-5 bg-white rounded border-0',style={'margin':'5px'})
def df_kom_func(komoditi):
    tes_aja = df_kom[df_kom['Muatan REV'] == komoditi].reset_index(drop=True)
    tes_aja = tes_aja.groupby(by=['Y_M','Tujuan Provinsi 2'], as_index=False)['Muatan REV 2'].count()
    map_date = [] 
    for i in range(df_kom['Year'].min(),df_kom['Year'].max()+1):
        for j in  ['01','02','03','04','05','06','07','08','09','10','11','12']:
            map_date.append('{i}-{j}'.format(i=i,j=j))
    map_date = pd.Series(map_date)
    date_missing = pd.DataFrame({'Y_M':[],'Tujuan Provinsi 2':[],'Muatan REV 2':[]})
    for i in map_date[~map_date.isin(tes_aja['Y_M'])]:
        date_missing.loc[len(date_missing)] = [i, np.NaN,np.NaN]
    tes_aja = pd.concat([tes_aja,date_missing]).sort_values(by='Y_M', ignore_index=True)
    tes_aja = tes_aja.rename(columns={'Y_M':'Year-Month','Tujuan Provinsi 2':'Provinsi','Muatan REV 2':'Jumlah KAB'})
    return tes_aja 
def df_kom_kat_func(kat_muatan):
    tes_juga = df_kom_kat[df_kom_kat['Kategori Muatan']==kat_muatan].reset_index(drop=True)
    tes_juga = tes_juga.groupby(by=['Y_M','Tujuan Provinsi 2'], as_index=False)['Muatan REV 2'].count()
    map_date = [] 
    for i in range(df_kom_kat['Year'].min(),df_kom_kat['Year'].max()+1):
        for j in  ['01','02','03','04','05','06','07','08','09','10','11','12']:
            map_date.append('{i}-{j}'.format(i=i,j=j))
    map_date = pd.Series(map_date) 
    date_missing = pd.DataFrame({'Y_M':[],'Tujuan Provinsi 2':[],'Muatan REV 2':[]})
    for i in map_date[~map_date.isin(tes_juga['Y_M'])]:
        date_missing.loc[len(date_missing)] = [i, np.NaN,np.NaN]
    tes_juga = pd.concat([tes_juga,date_missing]).sort_values(by='Y_M', ignore_index=True)
    tes_juga = tes_juga.rename(columns={'Y_M':'Year-Month','Tujuan Provinsi 2':'Provinsi','Muatan REV 2':'Jumlah KAB'})
    return tes_juga 
def df_komoditi(tahun): 
    return sql_df("""
        SELECT `Tanggal`, `Muatan REV`, `Kategori Muatan`,`Asal Provinsi`,`Tujuan Provinsi` FROM jt.master{th}
        ORDER BY `Tanggal`
        """.format(th=tahun)) 
# PATH_SOURCE_DATA geojohnson
path = 'D:\Fix\Data\indo_gjsn.json'
import json 
with open(path) as user_file:
    gjsn = json.load(user_file) 
df_map = pd.DataFrame({"Column": pd.json_normalize(gjsn["features"])["properties.state"]}).assign(Columnnext=lambda d: d["Column"].str.len())
df_kom_20 = df_komoditi(2020)
df_kom_21 = df_komoditi(2021) 
df_kom_22 = df_komoditi(2022)
df_kom_23 = df_komoditi(2023) 
df_kom = pd.concat([df_kom_20,df_kom_21,df_kom_22,df_kom_23]).sort_values(by='Tanggal', ignore_index=True)
df_kom['Tujuan Provinsi 2'] = pd.Series([i.title() for i in df_kom['Tujuan Provinsi']])
df_kom['Asal Provinsi 2'] = pd.Series([i.title() for i in df_kom['Asal Provinsi']])
plotly_prov_dict = {'Di Yogyakarta' : 'Yogyakarta','Dki Jakarta':'Jakarta Raya','Sumatra Utara': 'Sumatera Utara','Sumatra Barat':'Sumatera Barat','Sumatra Selatan':'Sumatera Selatan','Kepulauan Bangka Belitung':'Bangka-Belitung'} 
df_kom['Tujuan Provinsi 2'].replace(plotly_prov_dict, inplace=True)
df_kom['Asal Provinsi 2'].replace(plotly_prov_dict, inplace=True)
df_kom.drop(columns='Tujuan Provinsi', inplace=True)
df_kom.drop(columns='Asal Provinsi', inplace=True)
df_kom['Year'] = df_kom['Tanggal'].dt.year
df_kom['Month'] = df_kom['Tanggal'].dt.month 
df_kom_persebaran = df_kom.copy() 
df_kom_persebaran = df_kom_persebaran[df_kom_persebaran['Tujuan Provinsi 2']!='Tidak Diketahui']
df_kom_persebaran = df_kom_persebaran[df_kom_persebaran['Asal Provinsi 2']!='Tidak Diketahui']
df_kom_persebaran = df_kom_persebaran[df_kom_persebaran['Muatan REV']!='TIDAK DIKETAHUI' ].reset_index(drop=True)
df_kom_persebaran['Y_M'] = df_kom_persebaran['Tanggal'].dt.strftime('%Y-%m')
df_kom_persebaran['Muatan REV 2'] = df_kom_persebaran['Muatan REV'].copy() 
df_kom_persebaran = df_kom_persebaran[~df_kom_persebaran['Muatan REV'].isin(['KOSONG'])].reset_index(drop=True)
df_kom = df_kom[df_kom['Tujuan Provinsi 2']!='Tidak Diketahui' ].reset_index(drop=True)
df_kom = df_kom[df_kom['Asal Provinsi 2']!='Tidak Diketahui' ].reset_index(drop=True)
df_kom = df_kom[df_kom['Muatan REV']!='TIDAK DIKETAHUI' ].reset_index(drop=True)
df_kom['Y_M'] = df_kom['Tanggal'].dt.strftime('%Y-%m')
df_kom['Muatan REV 2'] = df_kom['Muatan REV'].copy()
df_kom = df_kom[~df_kom['Muatan REV'].isin(['KOSONG'])].reset_index(drop=True)
df_kom_kat = df_kom.copy()
df_kom_kat2 = df_kom.copy()
dict_muatan = {} 
for i in df_kom['Kategori Muatan'].unique():
    dict_muatan[i] = df_kom[df_kom['Kategori Muatan']==i]['Muatan REV'].value_counts().head().index.to_list()
list_top_muatan = []
for i in dict_muatan.values():
    for j in i:
        list_top_muatan.append(j)
df_kom = df_kom[df_kom['Muatan REV'].isin(list_top_muatan)].reset_index(drop=True)
df_kom_persebaran = df_kom_persebaran[df_kom_persebaran['Muatan REV'].isin(list_top_muatan)].reset_index(drop=True)
df_kom_fin = {}
for i in list_top_muatan:
    df_kom_fin[i] = df_kom_func(i)
df_kom_kat_fin = {} 
for i in df_kom_kat['Kategori Muatan'].unique():
    df_kom_kat_fin[i] = df_kom_kat_func(i)
map_date = [] 
for i in range(df_kom_kat['Year'].min(),df_kom_kat['Year'].max()+1):
    for j in  ['01','02','03','04','05','06','07','08','09','10','11','12']:
        map_date.append('{i}-{j}'.format(i=i,j=j))
map_date = pd.Series(map_date).iloc[:n_ticks]
bahan_forecast = {'BARANG LAINNYA':{},'BARANG KEBUTUHAN POKOK HASIL PERTANIAN':{},'BARANG PENTING':{},'BARANG KEBUTUHAN POKOK HASIL INDUSTRI':{},'BARANG KEBUTUHAN POKOK HASIL PETERNAKAN DAN PERIKANAN':{}}
for h,i in df_kom_kat_fin.items(): 
    for y in i['Provinsi'].unique(): 
        df_date_missing = pd.DataFrame({'Year-Month':[],'Provinsi':[],'Jumlah KAB':[]}) 
        for k in map_date[~map_date.isin(i[i['Provinsi']==y]['Year-Month'])]: 
            df_date_missing.loc[len(df_date_missing)] = [k,y,0]
        complete_date = pd.concat([i[i['Provinsi']==y],df_date_missing ]).sort_values(by='Year-Month', ignore_index=True) 
        complete_date['Year-Month'] = pd.to_datetime(complete_date['Year-Month']).dt.strftime('%Y-%m')
        bahan_forecast[h][y] = complete_date 
hasil_forecast_komoditi= {'BARANG LAINNYA':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG KEBUTUHAN POKOK HASIL PERTANIAN':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG PENTING':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG KEBUTUHAN POKOK HASIL INDUSTRI':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG KEBUTUHAN POKOK HASIL PETERNAKAN DAN PERIKANAN':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB'])}
for i in bahan_forecast.keys():
    del bahan_forecast[i][np.NaN] 
model_map = StatsForecastAutoARIMA()
for i,j in bahan_forecast.items():
    for k in j.keys():
        series_map = TimeSeries.from_dataframe(bahan_forecast[i][k], time_col='Year-Month',value_cols='Jumlah KAB',fill_missing_dates=True, freq='MS')
        model_map.fit(series_map)
        pred_map=model_map.predict(n=6, num_samples=500) 
        forecast_komoditi = pd.DataFrame( [0 if math.isnan(i) else int(i) for i in round(pred_map.quantile_df(0.5)['Jumlah KAB_0.5'])], index=pred_map.quantile_df(0.5).index).rename(columns={0:'Jumlah KAB'}).reset_index()
        for z in forecast_komoditi['Jumlah KAB'].index:
            if forecast_komoditi['Jumlah KAB'].iloc[z] <=0: 
                forecast_komoditi['Jumlah KAB'].iloc[z] = 0 
        forecast_komoditi['Year-Month'] = pd.to_datetime(forecast_komoditi['Year-Month']).dt.strftime('%Y-%m') 
        forecast_komoditi['Provinsi'] = k
        hasil_forecast_komoditi[i] = pd.concat([hasil_forecast_komoditi[i],forecast_komoditi]) 
final_df_komoditi = {'BARANG LAINNYA':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG KEBUTUHAN POKOK HASIL PERTANIAN':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG PENTING':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG KEBUTUHAN POKOK HASIL INDUSTRI':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG KEBUTUHAN POKOK HASIL PETERNAKAN DAN PERIKANAN':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB'])}
bahan_forecast = {'BARANG LAINNYA':{},'BARANG KEBUTUHAN POKOK HASIL PERTANIAN':{},'BARANG PENTING':{},'BARANG KEBUTUHAN POKOK HASIL INDUSTRI':{},'BARANG KEBUTUHAN POKOK HASIL PETERNAKAN DAN PERIKANAN':{}}
for h,i in df_kom_kat_fin.items(): 
    for y in i['Provinsi'].unique(): 
        df_date_missing = pd.DataFrame({'Year-Month':[],'Provinsi':[],'Jumlah KAB':[]}) 
        for k in map_date[~map_date.isin(i[i['Provinsi']==y]['Year-Month'])]: 
            df_date_missing.loc[len(df_date_missing)] = [k,y,0] 
        complete_date = pd.concat([i[i['Provinsi']==y],df_date_missing]).sort_values(by='Year-Month', ignore_index=True) 
        complete_date['Year-Month'] = pd.to_datetime(complete_date['Year-Month']).dt.strftime('%Y-%m')
        bahan_forecast[h][y] = complete_date 
hasil_forecast_komoditi= { 'BARANG LAINNYA':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG KEBUTUHAN POKOK HASIL PERTANIAN':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG PENTING':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG KEBUTUHAN POKOK HASIL INDUSTRI':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG KEBUTUHAN POKOK HASIL PETERNAKAN DAN PERIKANAN':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB'])}
model_map = StatsForecastAutoARIMA()
for i,j in bahan_forecast.items():
    for k in j.keys(): 
        series_map = TimeSeries.from_dataframe(bahan_forecast[i][k], time_col='Year-Month',value_cols='Jumlah KAB',fill_missing_dates=True, freq='MS')
        model_map.fit(series_map)
        pred_map=model_map.predict(n=6, num_samples=500) 
        forecast_komoditi = pd.DataFrame( [0 if math.isnan(i) else int(i) for i in round(pred_map.quantile_df(0.5)['Jumlah KAB_0.5'])], index=pred_map.quantile_df(0.5).index).rename(columns={0:'Jumlah KAB'}).reset_index()
        for z in forecast_komoditi['Jumlah KAB'].index:
            if forecast_komoditi['Jumlah KAB'].iloc[z] <=0:
                forecast_komoditi['Jumlah KAB'].iloc[z] = 0 
        forecast_komoditi['Year-Month'] = pd.to_datetime(forecast_komoditi['Year-Month']).dt.strftime('%Y-%m') 
        forecast_komoditi['Provinsi'] = k
        hasil_forecast_komoditi[i] = pd.concat([hasil_forecast_komoditi[i],forecast_komoditi])
final_df_komoditi = {'BARANG LAINNYA':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG KEBUTUHAN POKOK HASIL PERTANIAN':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG PENTING':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG KEBUTUHAN POKOK HASIL INDUSTRI':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB']),'BARANG KEBUTUHAN POKOK HASIL PETERNAKAN DAN PERIKANAN':pd.DataFrame(columns=['Year-Month','Provinsi','Jumlah KAB'])}
for i in df_kom_kat_fin.keys(): 
    final_df_komoditi[i] = pd.concat([df_kom_kat_fin[i],hasil_forecast_komoditi[i]]).sort_values(by='Year-Month', ignore_index=True)
for i in final_df_komoditi.keys(): 
    final_df_komoditi[i]['Year-Month'] = pd.to_datetime(final_df_komoditi[i]['Year-Month']).dt.strftime('%Y-%m')
for i in final_df_komoditi.keys(): 
    final_df_komoditi[i]['Jumlah KAB'] = final_df_komoditi[i]['Jumlah KAB'].replace({np.NaN:0}).astype(int) 
hghg = list(df_kom['Year'].unique()) 
uniqueprov= [] 
for i in final_df_komoditi.keys():
    for j in final_df_komoditi[i]['Provinsi'].unique():
        uniqueprov.append(j)
uniqueprov = list(pd.Series(uniqueprov).unique())
persebaran_component = dbc.Card([dbc.Row([dbc.Col([html.Span('Pilih Provinsi :'),dcc.Dropdown(id='opt_provinsi_persebaran', options=uniqueprov, value='Jawa Timur')]),dbc.Col([html.Span('Pilih Tahun :'),dcc.Dropdown(id='opt_tahun_persebaran', options=[hghg],value='2022')])]),dbc.Row([dcc.Graph(id='graph_persebaran_muatan')])],class_name='shadow p-3 mb-2 bg-white rounded border-0',style={'margin':'5px'}) 
opt_kat_muatan = list(df_kom_kat['Kategori Muatan'].unique())
viz_map_opt = {'Kategori Muatan':opt_kat_muatan, 'Muatan':list_top_muatan}
muatan_component = dbc.Card([dbc.Row([dbc.Col([html.Span('Pilih Variable 1 :'),dcc.Dropdown(id='opt_kat_mut', options=['Kategori Muatan', 'Muatan'], value='Kategori Muatan')]),dbc.Col([html.Span('Pilih Variable 2 :'),dcc.Dropdown(id='final_opt', value='BARANG PENTING')])]),dbc.Row([dcc.Graph(id='graph_muatan')])],class_name='shadow p-3 mb-2 bg-white rounded border-0',style={'margin':'5px'}) 
# PATH_SOURCE_DATA rekapitulasi capaian pengawasan
smm_header = pd.read_excel("D:\Fix\Data\Penilaian SMM.xlsx", sheet_name='Rekap').iloc[2:3].values[0] 
bulan_data_smm = ['01','02','03','04','05','06','07','08','09','10', '11'] 
filter_uppkb = list(np.sort(df['Uppkb'].unique())) 
data_smm = {} 
smm_col =['UPPKB','PRODUKTIFITAS Nasional','IMP. SOP','Konsistensi SOP','Mutu Kinerja/Penilaian Kinerja','Target 100% dalam 1 Bulan Berjalan','Jumlah Realisasi Pendataan Bulan Berjalan','Persentase Pecapaian Target Bulan Berjalan'] 
for i in bulan_data_smm:  
    # PATH_SOURCE_DATA rekapitulasi capaian pengawasan looping
    data_smm[i]=pd.read_excel("D:\Fix\Data\Penilaian SMM.xlsx" .format(i), sheet_name='Rekap').iloc[3:].reset_index(drop=True)
    data_smm[i].columns = smm_header
    data_smm[i] = data_smm[i][['UPPKB','PRODUKTIFITAS Nasional','IMP. SOP','Konsistensi SOP','Mutu Kinerja/Penilaian Kinerja','Persentase Pecapaian Target Bulan Berjalan','Target 100% dalam 1 Bulan Berjalan','Jumlah Realisasi Pendataan Bulan Berjalan']]
    k = pd.Series(data_smm[i]['UPPKB'].unique()) 
for i,j in data_smm.items(): 
    data_smm[i] = j.set_index('UPPKB') 
df_sort_month = pd.DataFrame({'num': ['01','02','03','04','05','06','07','08','09','10','11','12'],'num2':[1,2,3,4,5,6,7,8,9,10,11,12], 'month':['January','February','March','April','May','June','July','August','September','October','November','December'],'bulan':['Januari','Februari','Maret','April','Mei','Juni','Juli','Agustus','September','Oktober','November','Desember']}) 
rapor_filter_bulan = list(df_sort_month[df_sort_month['month'].isin(np.sort(df[df['Tahun']==2023]['Bulan'].unique()))].sort_values(by='num')['month'] )  + ['all month']
unique_smm_uppkb = [] 
for i in data_smm.keys():
    for j in data_smm[i].index: 
        unique_smm_uppkb.append(j)
unique_smm_uppkb = pd.Series(unique_smm_uppkb).drop_duplicates().sort_values().reset_index(drop=True)
filter_uppkb = unique_smm_uppkb 
def rapor_kab_masuk (selected_uppkb,selected_bulan): 
    a_ = df[ (df['Tahun']==2023) &(df['Bulan']==selected_bulan) &(df['Uppkb']==selected_uppkb) ].drop(columns=['Uppkb','Tahun','Bulan','Status pengoperasian','Selisih','Bulan2','Date','Year_Month'] ).T.reset_index().set_axis(['Attribute','Jumlah KAB'],axis=1)
    return a_ 
def rapor_kab_masuk_all (selected_uppkb): 
    b_ = df[ (df['Tahun']==2023) &(df['Uppkb']==selected_uppkb)].drop(columns=['Uppkb','Tahun','Bulan','Status pengoperasian','Selisih','Bulan2','Date','Year_Month'] ).sum().reset_index().set_axis(['Attribute','Jumlah KAB'],axis=1) 
    return b_
jt_distinct = sql_df("""SELECT DISTINCT(`Nama JT`) FROM jt.master2023""")
filter_uppkb_upper = pd.DataFrame(map(lambda i : i.upper(), filter_uppkb), columns=['uppkb']) 
df_company = {}
for i in filter_uppkb_upper['uppkb']:
    b = [] 
    for j in rapor_filter_bulan[:-1]:
        raport_perusahaan_mentah = sql_df("""
WITH cte_comp  as (
SELECT `Nama Grup`,`Nama JT`, `Kategori`,`Bulan` FROM jt.master2023
WHERE(`Nama JT` = '{a}') & (`Kategori` = 'Perusahaan') & (`Bulan` = '{b}') & (`Status Melanggar/Tidak` = 'Melanggar')
)
SELECT `Nama Grup`, COUNT(`Nama Grup`) as 'Jumlah' FROM cte_comp
GROUP BY `Nama Grup` 
ORDER BY 2 DESC LIMIT 5 ; 
""".format(a=i,b=df_sort_month[df_sort_month['month']== j]['bulan'].values[0])) 
        b.append( raport_perusahaan_mentah )
    df_company[i] = b 
df_company2 = {} 
for i in filter_uppkb_upper['uppkb']:
    raport_perusahaan_mentah2 = sql_df("""
        WITH cte_comp  as (
        SELECT `Nama Grup`,`Nama JT`, `Kategori`,`Bulan` FROM jt.master2023
        WHERE(`Nama JT` = '{a}') & (`Kategori` = 'Perusahaan') & (`Status Melanggar/Tidak` = 'Melanggar')
        )
        SELECT `Nama Grup`, COUNT(`Nama Grup`) as 'Jumlah' FROM cte_comp
        GROUP BY `Nama Grup`
        ORDER BY 2 DESC LIMIT 5 ;""".format(a=i))
    df_company2[i] = raport_perusahaan_mentah2 
bucket_kat_pel = []
for i in sql_df("""SELECT DISTINCT `Kategori %` FROM jt.master2023;""")['Kategori %']:
    if i == '0':
        pass  
    else:
        bucket_kat_pel.append(i)
unique_kat_pel=pd.DataFrame(bucket_kat_pel, columns=['Kategori %'])
# PATH_SOURCE_DATA rekapitulasi capaian pengawasan
Jan_ = pd.read_excel("D:\Fix\Data\Penilaian SMM.xlsx", sheet_name="Produktifitas")
vcols = Jan_.iloc[4:5].values[0] 
bulan_trend = 'September' 
angka_bulan_trend = 9 
df_t = {} 
for i in df_sort_month['bulan'][:df_sort_month[df_sort_month['bulan']==bulan_trend]['num2'].values[0]]:
    # PATH_SOURCE_DATA rekapitulasi capaian pengawasan looping
    y = pd.read_excel("D:\Fix\Data\Penilaian SMM.xlsx", sheet_name="Produktifitas".format(a=i)).iloc[:,:49]
    y.columns = vcols 
    df_t[i] = y 
df_t3 = {} 
for j in filter_uppkb: 
    df_t2 = pd.DataFrame() 
    for i in df_sort_month.iloc[:angka_bulan_trend]['bulan']:
        df_t2=pd.concat([df_t2,df_t[i][df_t[i]['BPTD/UPPKB']==j]]) 
    df_t3[j] = df_t2[['KAB TERDATA/ BULAN ','% PENGAWASAN']].reset_index(drop=True)
for i in df_t3.keys():  
    df_t3[i]['Bulan'] =  df_sort_month.iloc[:angka_bulan_trend]['bulan']
for i in df_t3.keys(): 
    df_t3[i]['% PENGAWASAN'] = [round((i*100),2) for i in df_t3[i]['% PENGAWASAN']] 
df_eval = df[(df['Tahun']==2023)&(df['Uppkb'].isin(filter_uppkb))].reset_index(drop=True)[['Uppkb','Bulan','Kend. masuk','Melanggar','Peringatan','Tilang','Kepolisian','Penundaan perjalanan','Putar balik','Transfer muatan','Penandaan kelebihan dimensi']]
eval_col = ((df_eval['Tilang'] + df_eval['Peringatan'] + df_eval['Kepolisian'] + df_eval['Penundaan perjalanan'] + df_eval['Putar balik'] + df_eval['Penandaan kelebihan dimensi'] + df_eval['Transfer muatan']) / df_eval['Melanggar'] * 100).replace({np.NaN:0})
eval_col=pd.Series([round(i,2) for i in eval_col])
df_eval['Persentase'] = eval_col 
df_eval_table = df_eval[df_eval['Uppkb'] == 'Datae'] 
eval_unique_month = list(df_eval_table['Bulan'].unique())
eval_index = df_eval.columns[2:] 
eval_bucket = [] 
eval_df = pd.DataFrame() 
df_eval_table__= df_eval_table.T.iloc[2:].rename(columns=df_eval_table['Bulan']).reset_index().rename(columns={'index':''})
df_eval_table__[''] = ['Kend. Masuk', 'Melanggar','Peringatan','Tilang','Kepolisian',
 'Penundaan Perjalanan','Putar Balik','Transfer Muatan', 'Penandaan Kelebihan Dimensi', 'Persentase' ]
color_eval_ = ['#D8D9DA','#FF8080','#FFCF96','#F6FDC3','#CDFAD5','#8EACCD','#E9B384','#BEADFA','#ECE3CE','red']
color_map = {}  
for i,j in zip(df_eval_table__[''], color_eval_):
    color_map.update({i:j}) 
df_eval_table__['color'] = df_eval_table__[''].map(color_map) 
lay_main_title_rapor=dmc.Grid(children=[dmc.Col([dmc.SimpleGrid([dmc.Text(id='rapor_title',weight=500,align='left',style={"fontSize": 33}, color='dark.9'),], cols=1, spacing=0)],span=12),],gutter='xl')
component_filter_rapor = dbc.Card([dbc.Row([dbc.Col([html.Span('Pilih Uppkb :'),html.Br(),dcc.Dropdown(id='rapor_filter_uppkb',options=filter_uppkb, value=filter_uppkb[1])]),dbc.Col([html.Span('Pilih Bulan :'),html.Br(),dcc.Dropdown(id='rapor_filter_bulan',options=rapor_filter_bulan, value=rapor_filter_bulan[-1])])])],class_name='shadow p-3 mb-2 bg-white rounded border-0',style={'margin':'5px'})
comp_rapor_pie = dbc.Card([dbc.Row([dmc.Space(h='sm'),dcc.Graph(id='viz_pie')])],class_name='shadow p-3 mb-2 bg-white rounded border-0',style={'margin':'5px'})
list_ops_penin_penang = list(['Pelanggaran','Penindakan', 'Penanganan'])
comp_rapor_penindakan_penanganan= dbc.Card([dbc.Row([dbc.Row([dcc.Dropdown(id='rapor_ops_penin_penang', options=list_ops_penin_penang, value=list_ops_penin_penang[0])]),dbc.Row([dcc.Graph(id='viz_penindakan_penanganan2')])])],class_name='shadow p-3 mb-2 bg-white rounded border-0',style={'margin':'5px'})
component_kat_pel = dbc.Card([dbc.Row([dmc.Space(h='sm'),dcc.Graph(id='bar_kat_pel')])],class_name='shadow p-3 mb-2 bg-white rounded border-0',style={'margin':'5px'}) 
title_rapor_pie = dmc.Center(dmc.Text('KAB Masuk',weight=500,align='center',style={'width':'100%',"fontSize": title_font_size}, color=title_font_color))
title_rapor_resume = dmc.Center(dmc.Text(id='title_pel_penin_pena',weight=500,align='center',style={'width':'100%',"fontSize": title_font_size}, color=title_font_color))
title_kat_pel = dmc.Center(dmc.Text('Kelompok Pelanggaran Daya Angkut',weight=500,align='center',style={'width':'100%',"fontSize": title_font_size}, color=title_font_color))
span_val = [4,4,4] 
wrapper_main_1 = dmc.Grid(children=[dmc.Col([comp_rapor_pie], span=span_val[0]),dmc.Col(comp_rapor_penindakan_penanganan, span=span_val[1]),dmc.Col(component_kat_pel, span= span_val[2])],gutter="xs",grow=True,align='stretch') 
wrapper_title_1 = dmc.Grid(children=[dmc.Col([title_rapor_pie], span=span_val[0]),dmc.Col(title_rapor_resume, span=span_val[1]),dmc.Col(title_kat_pel, span= span_val[2])],gutter="xs",grow=True,align='stretch',justify='center')
component_table_perusahaan = dbc.Card( [dbc.Row([dcc.Graph(id='tabel_perusahaan')])],class_name='shadow p-3 mb-2 bg-white rounded border-0',style={'margin':'5px'}) 
component_table_smm = dbc.Card( [dbc.Row([dcc.Graph(id='tabel_smm')])],class_name='shadow p-3 mb-2 bg-white rounded border-0',style={'margin':'5px'}) 
component_radar = dbc.Card([dbc.Row([dcc.Graph(id='rapor_radar')])],class_name='shadow p-3 mb-2 bg-white rounded border-0',style={'margin':'5px'}) 
title_perusahaan = dmc.Center(dmc.Text('Daftar Perusahaan Melanggar',weight=500,align='center',style={'width':'100%',"fontSize": title_font_size}, color=title_font_color))
title_smm = dmc.Center(dmc.Text('Resume Penerapan SMM',weight=500,align='center',style={'width':'100%',"fontSize": title_font_size}, color=title_font_color))
title_7_indikator = dmc.Center(dmc.Text('Indikator Penilaian',weight=500,align='center',style={'width':'100%',"fontSize": title_font_size}, color=title_font_color))
span_val = [4,4,4]
wrapper_main_2 = dmc.Grid(children=[dmc.Col([component_table_perusahaan], span=span_val[0]),dmc.Col(component_table_smm, span=span_val[1]),dmc.Col(component_radar, span= span_val[2])],gutter="xs",align='stretch',justify='center')
wrapper_title_2 = dmc.Grid(children=[dmc.Col([title_perusahaan], span=span_val[0]),dmc.Col(title_smm, span=span_val[1]),dmc.Col(title_7_indikator, span= span_val[2])],gutter="xs",align='stretch',justify='center')
title_eval = dmc.Center(dmc.Text('Evaluasi Kendaraan Melanggar',weight=500,align='center',style={'width':'100%',"fontSize": title_font_size}, color=title_font_color))  
table_trend_kab_peng = dbc.Card([dbc.Row([dcc.Graph(id='table_trend_kab_peng')])],class_name='shadow p-3 mb-2 bg-white rounded border-0',style={'margin':'5px'})
component_eval = dbc.Card([dbc.Row([dcc.Graph(id='eval_kab_melanggar')]),dbc.Row([dcc.Graph(id='table_trend_kab_peng')])],class_name='shadow p-3 mb-2 bg-white rounded border-0',style={'margin':'5px'})
title_kab_peng = dmc.Center(dmc.Text('Rekapitulasi Capaian Pengawasan',weight=500,align='center',style={'width':'100%',"fontSize": title_font_size}, color=title_font_color))
component_tp = dbc.Card( [dbc.Row([dcc.Graph(id='kab_pengawasan')])],class_name='shadow p-3 mb-2 bg-white rounded border-0', style={'margin':'5px'}) 
combined_tp = dbc.CardGroup([component_tp]) 
# PATH_SOURCE_DATA rekapitulasi capaian pengawasan
df_7_indicator = pd.read_excel("D:\Fix\Data\Penilaian SMM.xlsx",sheet_name='Indikator').iloc[:,:3]
df_7_indicator['Hasil'] = round(df_7_indicator['Hasil'] * 100,2)
def pp_viz(df_viz): 
    color_pie = ['#B9B4C7','#FAF0E6','#232D3F' ]
    penin_viz = go.Figure(go.Bar(x=df_viz['Jumlah KAB'],y=df_viz['Attribute'],orientation='h',text=[ '{a} (%{b})'.format(a=str(int(i)), b=j) for i,j in zip(df_viz['Jumlah KAB'],[ str(i) for i in round((df_viz['Jumlah KAB'] / df_viz['Jumlah KAB'].sum()) *100,2)])],textposition='outside',marker_color=color_pie[0]))
    penin_viz.update_traces(hoverinfo='skip')
    penin_viz.update_layout(autosize=True,height=250,margin=dict(l=10,r=10,b=20,t=20,pad=0),xaxis_range=[0,df_viz['Jumlah KAB'].max() + (df_viz['Jumlah KAB'].max()/2)],yaxis={'categoryorder':'array','categoryarray':df_viz.sort_values(by='Jumlah KAB', ascending=True)['Attribute'].to_list()}) 
    return penin_viz 
univ_var_cat={'Jenis Pelanggaran':['Daya angkut','Dimensi','Persyaratan teknis','Dokumen','Tata cara muat','Kelas jalan'],'Jenis Penindakan':['Peringatan','Tilang','Kepolisian','Tilang uppkb lain'],'Jenis Penanganan':['Penundaan perjalanan','Transfer muatan', 'Penandaan kelebihan dimensi','Putar balik','Penyesuaian persyaratan teknis', 'Penyesuaian tata cara muat']} 
color_map = {'MANDIRI':'#00ADB5','PENDAMPINGAN':'#FF2E63','ALL UPPKB':'#03C988'}
groupby_col = ['Uppkb','Tahun','Status pengoperasian']
elmo_logo = 'https://cdn0-production-images-kly.akamaized.net/INPb9iQpSDjq55-AbarsA57hL3k=/1280x720/smart/filters:quality(75):strip_icc():format(webp)/kly-media-production/medias/4070241/original/022205600_1656725645-FWjjrnuUcAA8-_e.jpg'
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
background = 'url(https://images.unsplash.com/photo-1557683316-973673baf926?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=3458&q=80)'
load_figure_template("CERULEAN")
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CERULEAN,dbc_css,dbc.icons.BOOTSTRAP,{"href": "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css","rel": "stylesheet","integrity": "sha512-iBBXm8fW90+nuLcSKlbmrPcLa0OT92xO1BIsZ+ywDWZCvqsWgccV3gFoRBv0z+8dLJgyAHIhR35VZc2oM/gI1w==","crossorigin": "anonymous","referrerpolicy": "no-referrer",}],suppress_callback_exceptions=True) 
app.layout = dbc.Container([dcc.Location(id='url'),dbc.Row([dbc.Col([lay_sidebar],width=1),dbc.Col([dbc.Container(id="rekapitulasi-content", children=[],class_name='dbc')] )],className="p-0")],fluid=True,class_name='dbc p-0',style={'background-color': '#F7F7F7'}) 
@app.callback(Output('line_indicator_figure','figure'),Output('line_figure','figure'),Output('univariate_figure_categoric','figure'),Output('univariate_figure_numeric','figure'),Output('corr_graph','figure'),Output('3d_graph','figure'),Output('table_wilcoxon', 'figure'),Input("filter_komparasi", "value"),Input('filter_tahun','value'),Input('line_ind_var','value'),Input('univariate_dropdown_categoric','value'),Input('univariate_dropdown_numeric_graph_type','value'),Input('univariate_dropdown_numeric_var','value'),Input('corr_option_x','value'),Input('corr_option_y','value'),Input('3d_option_x','value'),Input('3d_option_y','value'),Input('3d_option_z','value'),Input('var_wilcoxon','value'),Input('tahun_wilcoxon','value'),Input('viz_wilcoxon','value'))
def plotting(filter_komparasi,filter_tahun,line_indicator_var,univariate_dropdown_categoric,univariate_num_graph_type,univariate_num_var,corr_option_x,corr_option_y,_3d_option_x,_3d_option_y,_3d_option_z,wilcoxon_var_selected,wilcoxon_year_selected,wilcoxon_viz_opt):
    if filter_komparasi == [1]: 
        indicator_value_m = df[(df['Status pengoperasian']=='MANDIRI') & (df['Tahun'].isin(filter_tahun))][line_indicator_var].sum()
        indicator_delta_m = df[(df['Status pengoperasian']== 'MANDIRI') & (df['Tahun'].isin(filter_tahun[:-1]))][line_indicator_var].sum()
        indicator_value_p = df[(df['Status pengoperasian']=='PENDAMPINGAN') & (df['Tahun'].isin(filter_tahun))][line_indicator_var].sum()
        indicator_delta_p = df[(df['Status pengoperasian']== 'PENDAMPINGAN') & (df['Tahun'].isin(filter_tahun[:-1]))][line_indicator_var].sum()
        df_line_on = df[df['Tahun'].isin(filter_tahun)].groupby(by=['Status pengoperasian','Year_Month'], as_index=False)[line_indicator_var].sum()
        df_pel_p = pd.DataFrame(df[(df['Status pengoperasian']=='PENDAMPINGAN') & (df['Tahun'].isin(filter_tahun))][groupby_col+univ_var_cat[univariate_dropdown_categoric]].groupby(by=groupby_col).sum().reset_index().sum().iloc[3:]).reset_index().sort_values(by=0,ascending=False).rename(columns={'index':univariate_dropdown_categoric,0:'Jumlah KAB'})
        df_pel_p['Percentage'] = [str(round((i/df_pel_p['Jumlah KAB'].sum())* 100,1))+ ' %' for i in df_pel_p['Jumlah KAB']]
        df_pel_p.insert(0, 'Pengoperasian','PENDAMPINGAN') 
        df_pel_m = pd.DataFrame(df[(df['Status pengoperasian']=='MANDIRI') & (df['Tahun'].isin(filter_tahun))][groupby_col+univ_var_cat[univariate_dropdown_categoric]].groupby(by=groupby_col).sum().reset_index().sum().iloc[3:]).reset_index().sort_values(by=0,ascending=False).rename(columns={'index':univariate_dropdown_categoric,0:'Jumlah KAB'})
        df_pel_m['Percentage'] = [str(round((i/df_pel_m['Jumlah KAB'].sum())* 100,1))+ ' %' for i in df_pel_m['Jumlah KAB']]
        df_pel_m.insert(0, 'Pengoperasian','MANDIRI')
        df_pel_pm = pd.concat([df_pel_p,df_pel_m],ignore_index=True)
        df_univariate = df[(df['Tahun'].isin(filter_tahun))][[univariate_num_var,'Uppkb','Tahun','Status pengoperasian','Bulan']]
        desc_num_m = pd.DataFrame(df_univariate[df_univariate['Status pengoperasian'] == 'MANDIRI'][univariate_num_var].describe().reset_index()).rename(columns={'index':'Attribute Mandiri'}) 
        desc_num_p = pd.DataFrame(df_univariate[df_univariate['Status pengoperasian'] == 'PENDAMPINGAN'][univariate_num_var].describe().reset_index()).rename(columns={'index':'Attribute Pendampingan'}) 
        df_corr = df[(df['Tahun'].isin(filter_tahun))][[corr_option_x,corr_option_y,'Uppkb','Tahun','Status pengoperasian','Bulan']]
        pears_on_m = round(df_corr[df_corr['Status pengoperasian'] == 'MANDIRI'].iloc[:,0].corr(df_corr[df_corr['Status pengoperasian'] == 'MANDIRI'].iloc[:,1]),3)
        pears_on_p = round(df_corr[df_corr['Status pengoperasian'] == 'PENDAMPINGAN'].iloc[:,0].corr(df_corr[df_corr['Status pengoperasian'] == 'PENDAMPINGAN'].iloc[:,1]),3)
        df_3d = df[(df['Tahun'].isin(filter_tahun))] 
        line_indicator_fig = px.line(df_line_on, x='Year_Month', y=line_indicator_var, markers=True, color='Status pengoperasian',color_discrete_map=color_map)
        line_indicator_fig.add_hline(y=df_line_on[df_line_on['Status pengoperasian']=='MANDIRI'][line_indicator_var].mean(), line_dash='dot', line_color=color_map['MANDIRI'])
        line_indicator_fig.add_hline(y=df_line_on[df_line_on['Status pengoperasian']=='PENDAMPINGAN'][line_indicator_var].mean(), line_dash='dot',line_color=color_map['PENDAMPINGAN'])
        line_indicator_fig.update_layout(autosize=True,height=300,margin=dict(l=0,r=0,b=0,t=0,pad=0),showlegend=False)
        line_indicator_fig.update_traces(line=dict(width=3),marker=dict(size=8))
        indicator_num_size = 25 
        indicator_font_size = 18
        indicator_delta_size = 15
        indicator_fig = go.Figure()
        indicator_fig.add_trace(go.Indicator(mode = "number+delta",value = indicator_value_m,number = {'suffix': " KAB","valueformat": ",","font":{"size":indicator_num_size,'color':color_map['MANDIRI']}},delta = {'position': "top", 'reference': indicator_delta_m,"valueformat": ",","font":{"size":indicator_delta_size}},domain = {'row': 0, 'column': 0},title = {'text':'UPPKB MANDIRI','font':{'size':indicator_font_size,'color':color_map['MANDIRI']}}))
        indicator_fig.add_trace(go.Indicator(mode = "number+delta",value = indicator_value_p,number = {'suffix': " KAB","valueformat": ",","font":{"size":indicator_num_size,'color':color_map['PENDAMPINGAN']}},delta = {'position': "top", 'reference': indicator_delta_p,"valueformat": ",","font":{"size":indicator_delta_size}},domain = {'row': 1, 'column': 0},title = {'text':'UPPKB PENDAMPINGAN','font':{'size':indicator_font_size,'color':color_map['PENDAMPINGAN']}}))
        indicator_fig.update_layout(grid = {'rows': 2, 'columns': 1, 'pattern': "independent"})
        indicator_fig.update_layout(autosize=True,height=240,margin=dict(l=0,r=0,b=0,t=60,pad=0)) 
        fig_univ_cat = px.bar(data_frame=df_pel_pm,x = 'Jumlah KAB',y = univariate_dropdown_categoric,color= 'Pengoperasian',barmode='group',orientation='h',hover_data=['Percentage'],color_discrete_map=color_map)
        fig_univ_cat.update_layout(yaxis=dict(autorange="reversed"),xaxis_range=[0, df_pel_pm['Jumlah KAB'].max() + (df_pel_pm['Jumlah KAB'].max() * 0.10)],showlegend=False,autosize=True,height=300,margin=dict(l=0,r=0,b=0,t=30,pad=0))
        univ_fig_num = go.Figure()
        if univariate_num_graph_type == 'boxplot':
            univ_fig_num = px.box(df_univariate,x=univariate_num_var,orientation='h',hover_name='Uppkb',hover_data=['Tahun','Bulan'],color_discrete_map=color_map,color='Status pengoperasian').update_layout(showlegend=False,autosize=True,height=300,margin=dict(l=0,r=0,b=0,t=30,pad=0))
        elif univariate_num_graph_type == 'histogram':
            univ_fig_num = px.histogram(df_univariate,x= univariate_num_var,histnorm="percent",color='Status pengoperasian',color_discrete_map=color_map,hover_name='Uppkb',hover_data=['Tahun','Bulan']).update_layout(showlegend=False,autosize=True,height=300,margin=dict(l=0,r=0,b=0,t=30,pad=0))
        elif univariate_num_graph_type == 'violinplot':
            univ_fig_num = px.violin(df_univariate,x= univariate_num_var,orientation='h',hover_name='Uppkb',hover_data=['Tahun','Bulan'],color='Status pengoperasian',color_discrete_map=color_map,box=True,points='all').update_layout(showlegend=False,autosize=True,height=300,margin=dict(l=0,r=0,b=0,t=30,pad=0))
        else : 
            univ_fig_num.add_trace(go.Table(header=dict(values=list(desc_num_m.columns),align='left'),cells=dict(values=[desc_num_m['Attribute Mandiri'], desc_num_m[univariate_num_var]],align='left'),domain = {'row': 0, 'column': 0}))
            univ_fig_num.add_trace(go.Table(
                header=dict(values=list(desc_num_p.columns),align='left'),cells=dict(values=[desc_num_p['Attribute Pendampingan'], desc_num_p[univariate_num_var]],align='left'),domain = {'row': 0, 'column': 1}))
            univ_fig_num.update_layout(grid = {'rows': 1, 'columns': 2, 'pattern': "independent"},autosize=True,height=300,margin=dict(l=0,r=0,b=0,t=30,pad=0))
        fig_corr = px.scatter(df_corr,x=df_corr.iloc[:,0],y=df_corr.iloc[:,1],trendline='ols', marginal_x='histogram',marginal_y='box',color='Status pengoperasian',color_discrete_map=color_map,hover_name='Uppkb',hover_data=['Tahun','Bulan'],title= """
        Pearson's correlation (Mandiri) : {a} <br>Pearson's correlation (Pendampingan) : {b}
        """.format(a=pears_on_m, b=pears_on_p)).update_layout(xaxis_title=df_corr.iloc[:,0].name, yaxis_title=df_corr.iloc[:,1].name, height=350,showlegend=False, margin=dict(l=0,r=0,b=0,t=50,pad=0))
        fig_3d = px.scatter_3d(df_3d,x=_3d_option_x,y=_3d_option_y,z=_3d_option_z,color='Status pengoperasian',color_discrete_map=color_map,hover_data=['Tahun','Bulan'],hover_name='Uppkb').update_layout(showlegend=False, height=350,margin=dict(l=0,r=0,b=0,t=0,pad=0))
        if wilcoxon_viz_opt == 'Graph':
            wilcoxon_table = make_subplots(rows=2,cols=2,subplot_titles=("UPPKB Pendampingan tahun {th}".format(th=str(wilcoxon_year_selected-1)),"UPPKB Pendampingan tahun {th}".format(th=str(wilcoxon_year_selected)),"UPPKB Mandiri tahun {th}".format(th=str(wilcoxon_year_selected-1)),"UPPKB Mandiri tahun {th}".format(th=str(wilcoxon_year_selected))),shared_yaxes=True)
            wilcoxon_table.add_trace(go.Scatter(x=coba_pendampingan[opt_tahun_map_wilcoxon[wilcoxon_year_selected]]["Y_M"],y=coba_pendampingan[opt_tahun_map_wilcoxon[wilcoxon_year_selected]][wilcoxon_var_selected], line_color=color_map['PENDAMPINGAN']),row=1, col=1)
            wilcoxon_table.add_trace(go.Scatter(x=coba_pendampingan[opt_tahun_map_wilcoxon[wilcoxon_year_selected]+1]["Y_M"],y=coba_pendampingan[opt_tahun_map_wilcoxon[wilcoxon_year_selected]+1][wilcoxon_var_selected], line_color=color_map['PENDAMPINGAN']),row=1, col=2)
            wilcoxon_table.add_trace(go.Scatter(x=coba_mandiri[opt_tahun_map_wilcoxon[wilcoxon_year_selected]]["Y_M"],y=coba_mandiri[opt_tahun_map_wilcoxon[wilcoxon_year_selected]][wilcoxon_var_selected], line_color=color_map['MANDIRI']),row=2, col=1)
            wilcoxon_table.add_trace(go.Scatter(x=coba_mandiri[opt_tahun_map_wilcoxon[wilcoxon_year_selected]+1]["Y_M"],y=coba_mandiri[opt_tahun_map_wilcoxon[wilcoxon_year_selected]+1][wilcoxon_var_selected], line_color=color_map['MANDIRI']),row=2, col=2)
            wilcoxon_table.update_layout(height=700,title_text="Perbandingan Jumlah {var} tahun {th_before} - {th_after}".format(var=opt_map_wilcoxon[wilcoxon_var_selected], th_before=str(wilcoxon_year_selected-1), th_after=str(wilcoxon_year_selected)), showlegend=False)
            wilcoxon_table.update_yaxes(title_text="Jumlah {var}".format(var=opt_map_wilcoxon[wilcoxon_var_selected]), row=1, col=1)
            wilcoxon_table.update_yaxes(title_text="Jumlah {var}".format(var=opt_map_wilcoxon[wilcoxon_var_selected]),row=1, col=2)
            wilcoxon_table.update_yaxes(title_text="Jumlah {var}".format(var=opt_map_wilcoxon[wilcoxon_var_selected]), row=2, col=1)
            wilcoxon_table.update_yaxes(title_text="Jumlah {var}".format(var=opt_map_wilcoxon[wilcoxon_var_selected]),row=2, col=2)
            wilcoxon_table.update_xaxes(title_text='Bulan-Tahun', row=1, col=1)
            wilcoxon_table.update_xaxes(title_text='Bulan-Tahun', row=1, col=2)
            wilcoxon_table.update_xaxes(title_text='Bulan-Tahun', row=2, col=1)
            wilcoxon_table.update_xaxes(title_text='Bulan-Tahun', row=2, col=2)
            wilcoxon_table.add_hline(row=1, col=1, line_dash='dot', y=coba_pendampingan[opt_tahun_map_wilcoxon[wilcoxon_year_selected]][wilcoxon_var_selected].median())
            wilcoxon_table.add_hline(row=1, col=2, line_dash='dot', y=coba_pendampingan[opt_tahun_map_wilcoxon[wilcoxon_year_selected]+1][wilcoxon_var_selected].median())
            wilcoxon_table.add_hline(row=2, col=1, line_dash='dot', y=coba_mandiri[opt_tahun_map_wilcoxon[wilcoxon_year_selected]][wilcoxon_var_selected].median())
            wilcoxon_table.add_hline(row=2, col=2, line_dash='dot', y=coba_mandiri[opt_tahun_map_wilcoxon[wilcoxon_year_selected]+1][wilcoxon_var_selected].median())
            wilcoxon_table.update_traces(line=dict(width=3),marker=dict(size=8))
        else:
            wilcoxon_table = go.Figure()
            wilcoxon_table.add_trace(go.Table(header=dict(values=list(['Attribute Mandiri', 'Wilcoxon Statistical Test']),align='left'),cells=dict(values=[result_mandiri[opt_tahun_map_wilcoxon[wilcoxon_year_selected]][wilcoxon_var_selected]['Attribute'].to_list(),result_mandiri[opt_tahun_map_wilcoxon[wilcoxon_year_selected]][wilcoxon_var_selected]['Wilcoxon Statistical Test'].to_list() ],align='left'),domain = {'row': 0, 'column': 0}))
            wilcoxon_table.add_trace(go.Table( header=dict(values=['Attribute Pendampingan', 'Wilcoxon Statistical Test'],align='left'),cells=dict(values=[result_pendampingan[opt_tahun_map_wilcoxon[wilcoxon_year_selected]][wilcoxon_var_selected]['Attribute'].to_list(),result_pendampingan[opt_tahun_map_wilcoxon[wilcoxon_year_selected]][wilcoxon_var_selected]['Wilcoxon Statistical Test'].to_list()],align='left'),domain = {'row': 0, 'column': 1}))
            wilcoxon_table.update_layout(grid = {'rows': 1, 'columns': 2, 'pattern': "independent"},autosize=True,height=300,margin=dict(l=0,r=0,b=0,t=30,pad=0))
        return indicator_fig,line_indicator_fig,fig_univ_cat, univ_fig_num, fig_corr, fig_3d,wilcoxon_table 
    else: 
        indicator_value = df[df['Tahun'].isin(filter_tahun)][line_indicator_var].sum()
        indicator_delta = df[df['Tahun'].isin(filter_tahun[:-1])][line_indicator_var].sum()
        df_line_off = df[df['Tahun'].isin(filter_tahun)].groupby(by=['Year_Month'], as_index=False)[line_indicator_var].sum()
        df_pel_all = pd.DataFrame(df[df['Tahun'].isin(filter_tahun)][groupby_col+univ_var_cat[univariate_dropdown_categoric]].groupby(by=groupby_col).sum().reset_index().sum().iloc[3:]).reset_index().sort_values(by=0,ascending=False).rename(columns={'index':univariate_dropdown_categoric,0:'Jumlah KAB'})
        df_pel_all['Percentage'] = [str(round((i/df_pel_all['Jumlah KAB'].sum())* 100,1))+ ' %' for i in df_pel_all['Jumlah KAB']]
        df_univariate = df[(df['Tahun'].isin(filter_tahun))][[univariate_num_var,'Uppkb','Tahun','Status pengoperasian','Bulan']]
        desc_num = pd.DataFrame(df_univariate[univariate_num_var].describe().reset_index()).rename(columns={'index':'attribute'})   
        df_corr = df[(df['Tahun'].isin(filter_tahun))][[corr_option_x,corr_option_y,'Uppkb','Tahun','Status pengoperasian','Bulan']]
        pears_off = round(df_corr.iloc[:,0].corr(df_corr.iloc[:,1]),3)
        df_3d = df[(df['Tahun'].isin(filter_tahun))]
        line_indicator_fig = px.line(df_line_off, x='Year_Month', y=line_indicator_var, markers=True)
        line_indicator_fig.add_hline(y=df_line_off[line_indicator_var].mean(), line_dash='dot')
        line_indicator_fig.update_layout(autosize=True,height=300,margin=dict(l=0,r=0,b=0,t=0,pad=0),showlegend=False)
        line_indicator_fig.update_traces(line=dict(width=3,color=color_map['ALL UPPKB']),marker=dict(size=8, color=color_map['ALL UPPKB']))
        indicator_num_size = 25
        indicator_font_size = 18
        indicator_delta_size = 15 
        indicator_fig = go.Figure(go.Indicator(mode = "number+delta",value = indicator_value,number = {'suffix': " KAB","valueformat": ",","font":{"size":indicator_num_size,'color':color_map['ALL UPPKB']}},delta = {'position': "top", 'reference': indicator_delta,"valueformat": ",","font":{"size":indicator_delta_size}},domain = {'row': 0, 'column': 0},title = {'text':'ALL UPPKB','font':{'size':indicator_font_size, 'color':color_map['ALL UPPKB']}})).update_layout(autosize=True,height=240,margin=dict(l=0,r=0,b=10,t=60,pad=0))
        fig_univ_cat = px.bar(data_frame=df_pel_all,x = 'Jumlah KAB',y = univariate_dropdown_categoric, barmode='group',orientation='h',hover_data=['Percentage'])
        fig_univ_cat.update_layout(yaxis=dict(autorange="reversed"),xaxis_range=[0, df_pel_all['Jumlah KAB'].max() + (df_pel_all['Jumlah KAB'].max() * 0.10)]).update_traces(marker=dict(color=color_map['ALL UPPKB']))
        univ_fig_num = go.Figure()
        if univariate_num_graph_type == 'boxplot':
            univ_fig_num = px.box(df_univariate,x=univariate_num_var,orientation='h',hover_name='Uppkb',hover_data=['Tahun','Bulan','Status pengoperasian']).update_traces(marker=dict(color=color_map['ALL UPPKB']))
        elif univariate_num_graph_type == 'histogram':
            univ_fig_num = px.histogram(df_univariate,x= univariate_num_var, histnorm="percent",hover_name='Uppkb', hover_data=['Tahun','Bulan','Status pengoperasian']).update_traces(marker=dict(color=color_map['ALL UPPKB']))
        elif univariate_num_graph_type == 'violinplot':
            univ_fig_num = px.violin(df_univariate,x= univariate_num_var,orientation='h',hover_name='Uppkb', hover_data=['Tahun','Bulan','Status pengoperasian'],box=True,points='all').update_traces(marker=dict(color=color_map['ALL UPPKB']))
        else :
            univ_fig_num = go.Figure(data=[go.Table(header=dict(values=list(desc_num.columns),align='left'),cells=dict(values=[desc_num['attribute'], desc_num[univariate_num_var]],align='left'))])
            univ_fig_num.update_layout(title_text='All UPPKB')
        fig_corr = px.scatter(df_corr,x=df_corr.iloc[:,0],y=df_corr.iloc[:,1],trendline='ols', marginal_x='box',marginal_y='box',hover_name='Uppkb',hover_data=['Tahun','Bulan'],title="Pearson's correlation : {}".format(pears_off)).update_layout(xaxis_title=df_corr.iloc[:,0].name, yaxis_title=df_corr.iloc[:,1].name).update_traces(marker=dict(color=color_map['ALL UPPKB']))
        fig_3d = px.scatter_3d(df_3d,x=_3d_option_x, y=_3d_option_y, z=_3d_option_z,hover_data=['Tahun','Bulan'],hover_name='Uppkb').update_traces(marker=dict(color=color_map['ALL UPPKB']))
        if wilcoxon_viz_opt == 'Graph':
            wilcoxon_table = make_subplots(rows=1, cols=2,subplot_titles=("UPPKB tahun {th}".format(th=str(wilcoxon_year_selected-1)),"UPPKB tahun {th}".format(th=str(wilcoxon_year_selected))),shared_yaxes=True)
            wilcoxon_table.add_trace(go.Scatter(x=coba_all[opt_tahun_map_wilcoxon[wilcoxon_year_selected]]["Y_M"],y=coba_all[opt_tahun_map_wilcoxon[wilcoxon_year_selected]][wilcoxon_var_selected], line_color=color_map['ALL UPPKB']),row=1, col=1)
            wilcoxon_table.add_trace(go.Scatter(x=coba_all[opt_tahun_map_wilcoxon[wilcoxon_year_selected]+1]["Y_M"],y=coba_all[opt_tahun_map_wilcoxon[wilcoxon_year_selected]+1][wilcoxon_var_selected], line_color=color_map['ALL UPPKB']),row=1, col=2)
            wilcoxon_table.update_layout(height=350,title_text="Perbandingan Jumlah {var} tahun {th_before} - {th_after}".format(var=opt_map_wilcoxon[wilcoxon_var_selected], th_before=str(wilcoxon_year_selected-1), th_after=str(wilcoxon_year_selected)),showlegend=False)
            wilcoxon_table.update_yaxes(title_text="Jumlah {var}".format(var=opt_map_wilcoxon[wilcoxon_var_selected]), row=1, col=1)
            wilcoxon_table.update_yaxes(title_text="Jumlah {var}".format(var=opt_map_wilcoxon[wilcoxon_var_selected]),row=1, col=2)
            wilcoxon_table.update_xaxes(title_text='Bulan-Tahun', row=1, col=1)
            wilcoxon_table.update_xaxes(title_text='Bulan-Tahun', row=1, col=2)
            wilcoxon_table.add_hline(row=1, col=1, line_dash='dot', y=coba_all[opt_tahun_map_wilcoxon[wilcoxon_year_selected]][wilcoxon_var_selected].median())
            wilcoxon_table.add_hline(row=1, col=2, line_dash='dot', y=coba_all[opt_tahun_map_wilcoxon[wilcoxon_year_selected]+1][wilcoxon_var_selected].median())
            wilcoxon_table.update_traces(line=dict(width=3),marker=dict(size=8))
        else:
            wilcoxon_table = go.Figure() 
            wilcoxon_table = go.Figure(data=[go.Table(header=dict(values=['Attribute', 'Wilcoxon Statistical Test'],align='left'),cells=dict(values=[result_all[opt_tahun_map_wilcoxon[wilcoxon_year_selected]][wilcoxon_var_selected]['Attribute'].to_list(),result_all[opt_tahun_map_wilcoxon[wilcoxon_year_selected]][wilcoxon_var_selected]['Wilcoxon Statistical Test'].to_list()],align='left'))])
        return indicator_fig,line_indicator_fig,fig_univ_cat, univ_fig_num, fig_corr,fig_3d, wilcoxon_table
@app.callback(Output('forecasting_graph','figure'),Output('score_graph', 'figure'),Output('tabel_forecasting_graph','figure'),Output('final_opt','options'),Input('opt_kat_mut','value'))
def plotting_forecasting(selected_kat_mut):
    y_upper =  round(pred.quantile_df(0.9)['%Melanggar_0.9'],2)
    y_lower = round(pred.quantile_df(0.1)['%Melanggar_0.1'],2)
    main_color_line = '#186F65' 
    val_color_line = '#B5CB99'
    pred_color_line = '#B2533E'
    line_width = 4.5
    forecasting = go.Figure([go.Scatter(name='Upper Bound',x=pred.quantile_df(0.5)['%Melanggar_0.5'].index,y=y_upper, mode='lines',marker=dict(color="#444"),line=dict(width=0),showlegend=False),go.Scatter(name='Lower Bound',x=pred.quantile_df(0.5)['%Melanggar_0.5'].index,y=y_lower,marker=dict(color="#444"),line=dict(width=0),mode='lines', fillcolor='#DBE2EF', fill='tonexty', showlegend=False)]) 
    forecasting.update_layout(yaxis_title='Percentage',hovermode="x") 
    forecasting.add_traces(go.Scatter(name='Actual',x = training.pd_dataframe().index,y = round(training.pd_series(),2),showlegend=False,line=dict(width=line_width),line_color= main_color_line )) 
    forecasting.add_traces(go.Scatter(name='Actual',x = validation.pd_dataframe().index,y = round(validation.pd_series(),2),showlegend=False,line=dict(width=line_width),line_color= main_color_line )) 
    forecasting.add_traces(go.Scatter(name='Validation', x = pred_val.quantile_df(0.5)['%Melanggar_0.5'].index,y = round(pred_val.quantile_df(0.5)['%Melanggar_0.5'],2), showlegend=False,line=dict(width=line_width),line_color = val_color_line))
    forecasting.add_vrect(x0=pred.quantile_df(0.5)['%Melanggar_0.5'].index[0], x1=pred.quantile_df(0.5)['%Melanggar_0.5'].index[-1], annotation_text="Forecasting Results", annotation_position="top",fillcolor="#DBE2EF", opacity=0.2, line_width=2) 
    forecasting.add_vrect(x0=pred_val.quantile_df(0.5)['%Melanggar_0.5'].index[0], x1=pred_val.quantile_df(0.5)['%Melanggar_0.5'].index[-1], annotation_text="Model Validation", annotation_position="top",fillcolor='#FFFFFF', opacity=0.2, line_width=2) 
    forecasting.add_vrect(x0=training.pd_dataframe().index[0], x1=training.pd_dataframe().index[-1], annotation_text="Data Train", annotation_position="top",fillcolor='#FFFFFF', opacity=0.2, line_width=2) 
    forecasting.add_traces(  go.Scatter(name='Prediksi',x=pred.quantile_df(0.5)['%Melanggar_0.5'].index,y=round(pred.quantile_df(0.5)['%Melanggar_0.5'],2),mode='lines',line=dict(width=line_width),showlegend=False,line_color=pred_color_line )) 
    len_index=  len(pd.concat([pd.Series(training.pd_dataframe().index), pd.Series(validation.pd_dataframe().index),pd.Series(pred.quantile_df(0.5)['%Melanggar_0.5'].index)],ignore_index=True)) 
    len_nticks= len(data_monthly_nasional)
    n_ticks = data_monthly_nasional.iloc[:len_nticks].shape[0] + 6
    tickvals = np.linspace(0,len_index, num=n_ticks, endpoint=True)
    train_ticks = [] 
    for i in range(1,13):
        for j in range(data_weekly_nasional['Tahun'].min(),data_weekly_nasional['Tahun'].max()+1):
            train_ticks.append(pd.to_datetime("{j}-{i}".format(j=j,i=i)))
    train_ticks = pd.Series(train_ticks).sort_values(ignore_index=True).dt.strftime('%Y-%m')
    pred_ticks = [] 
    tahun_terakhir_train = int(train_ticks.iloc[-1][0:4])
    bulan_terakhir_train =int(train_ticks.iloc[-1][5:7])
    if (data_weekly_nasional['Tahun'].max() == tahun_terakhir_train ) & (bulan_terakhir_train == 12):
        for i in range(1,7): 
            pred_ticks.append(pd.to_datetime("{j}-{i}".format(j=data_weekly_nasional['Tahun'].max()+1,i=i)))
        pred_ticks = pd.Series(pred_ticks).sort_values(ignore_index=True).dt.strftime('%Y-%m')
    else: 
        for i in range(bulan_terakhir_train+1,13):
            pred_ticks.append(pd.to_datetime("{j}-{i}".format(j=data_weekly_nasional['Tahun'].max(),i=i)))
        pred_ticks = pd.Series(pred_ticks).sort_values(ignore_index=True).dt.strftime('%Y-%m')
    ticktext = pd.concat([train_ticks,pred_ticks], ignore_index=True) 
    forecasting.update_layout(xaxis = dict(tickmode = 'array',tickvals = tickvals,ticktext =ticktext,showgrid=False),yaxis = dict(showgrid=False))
    forecasting.update_layout(autosize=True,height=400,margin=dict(l=0,r=0,b=100,t=20,pad=10))
    rmse = round(np.sqrt(mean_squared_error(y_true=validation.pd_series(), y_pred=pred_val.quantile_df(0.5)[:len(validation.pd_series())])),2)
    mae = round(mean_absolute_error(y_true=validation.pd_series(), y_pred=pred_val.quantile_df(0.5)[:len(validation.pd_series())]), ndigits=2)
    mape = round(mean_absolute_percentage_error(y_true=validation.pd_series(), y_pred=pred_val.quantile_df(0.5)[:len(validation.pd_series())])*100, ndigits=2) 
    pred_val_mean = round(pred_val.quantile_df(0.5)['%Melanggar_0.5'].mean(), ndigits=2)
    val_mean = round(validation.pd_series().mean(), ndigits=2)
    pred_mean = round(pred.quantile_df(0.5)['%Melanggar_0.5'].mean() , ndigits=2)
    diff = round(abs(pred_val.quantile_df(0.5)['%Melanggar_0.5'].mean()-validation.pd_series().mean()), ndigits=2)
    score = pd.DataFrame({'Attribute':['RMSE','MAE','MAPE','Validation Mean', 'Prediction Validation Mean', 'Differences','Prediction Mean'], 'Value':[rmse,mae,mape,val_mean, pred_val_mean,diff, pred_mean]})
    score_forecasting = go.Figure()
    score_forecasting = go.Figure(data=[go.Table(header=dict(values=['Attribute', 'Value'],line_color='black',fill_color= '#337CCF',align='left',font=dict(color='white', size=14),height=30),cells=dict(values=[score['Attribute'].to_list(),score['Value'].to_list()],line_color='#F1F3F5',fill_color=[tabel_forecasting_.color],align='left',font=dict(color='black', size=14),height=30)),]).update_layout(autosize=True,height=300,margin=dict(l=0,r=0,b=30,t=10,pad=10))
    tabel_forecasting = go.Figure(data=[go.Table(header=dict( values=['Date', '% Melanggar'],line_color='black',fill_color= '#337CCF',align='left',font=dict(color='white', size=14),height=30),cells=dict( values=[tabel_forecasting_['Date'],tabel_forecasting_['% Melanggar']],line_color= [tabel_forecasting_.color],fill_color= [tabel_forecasting_.color],align='left',font=dict(color='black', size=14),height=30))]).update_layout(autosize=True,height=280,margin=dict(l=0,r=0,b=30,t=10,pad=10))
    return forecasting, score_forecasting , tabel_forecasting, viz_map_opt[selected_kat_mut]
@app.callback(Output('graph_muatan','figure'),Input('final_opt','value'))
def plot_map(selected_var_map): 
    color_continuous_scale = 'Sunsetdark'
    mapbox_style = "open-street-map"
    if pd.Series(list_top_muatan).isin([selected_var_map]).any() == True:
        map_max_range = df_kom_fin[selected_var_map]['Jumlah KAB'].max()
        fig_muatan = px.choropleth_mapbox(df_kom_fin[selected_var_map], geojson=gjsn,color="Jumlah KAB",locations="Provinsi", featureidkey="properties.state",center={"lat": -2.440095, "lon":  118.301787},mapbox_style=mapbox_style, zoom=4,animation_frame= 'Year-Month', color_continuous_scale=color_continuous_scale, range_color = (0,map_max_range))
        fig_muatan.update_traces(marker_line_width = 0.5, marker_line_color = 'white')
        fig_muatan.update_layout(height=600,margin={"r":20,"t":10,"l":30,"b":10}, showlegend=False)
        return fig_muatan 
    else:
        map_max_range_kat = final_df_komoditi[selected_var_map]['Jumlah KAB'].max()
        fig_komoditi = px.choropleth_mapbox(final_df_komoditi[selected_var_map], geojson=gjsn, color="Jumlah KAB",locations="Provinsi", featureidkey="properties.state",center={"lat": -2.440095, "lon":  118.301787}, mapbox_style=mapbox_style, zoom=4, animation_frame= 'Year-Month',color_continuous_scale=color_continuous_scale ,range_color = (0,map_max_range_kat)) 
        fig_komoditi.update_traces(marker_line_width = 0.5, marker_line_color = 'white')
        fig_komoditi.update_layout(height=600,margin={"r":20,"t":10,"l":30,"b":10}, showlegend=False) 
        return fig_komoditi 
color_pie = ['#BBBFCA','#E8E8E8','#232D3F' ]
def pp_viz(df_viz):
    penin_viz = go.Figure(go.Bar(x=df_viz['Jumlah KAB'],y=df_viz['Attribute'],orientation='h',text=[ '{a} (%{b})'.format(a=str(int(i)), b=j) for i,j in zip(df_viz['Jumlah KAB'],[ str(i) for i in round((df_viz['Jumlah KAB'] / df_viz['Jumlah KAB'].sum()) *100,2)])],textposition='outside', marker_color=color_pie[0],))
    penin_viz.update_traces(hoverinfo='skip')
    penin_viz.update_layout(autosize=True,height=255,margin=dict(l=0,r=0,b=10,t=10,pad=0),xaxis_range=[0,df_viz['Jumlah KAB'].max() + (df_viz['Jumlah KAB'].max()/2)],yaxis={'categoryorder':'array','categoryarray':df_viz.sort_values(by='Jumlah KAB', ascending=True)['Attribute'].to_list()})
    return penin_viz
@app.callback( Output('rapor_title', 'children'),Output('viz_pie','figure'),Output('title_pel_penin_pena', 'children'), Output('viz_penindakan_penanganan2','figure'),Output('bar_kat_pel','figure'), Output('tabel_perusahaan','figure'),Output('tabel_smm','figure'),Output('rapor_radar', 'figure'),Output('eval_kab_melanggar','figure'), Output('table_trend_kab_peng','figure'),Output('kab_pengawasan','figure'), Input('rapor_filter_uppkb','value'),Input('rapor_filter_bulan','value'), Input('rapor_ops_penin_penang','value'))
def rapor( filter_uppkb,filter_bulan,filter_penin_penang):
    u__ = filter_uppkb 
    b__ = filter_bulan  
    pelanggaran_list = ['Daya angkut','Dimensi','Persyaratan Teknis','Dokumen','Tata cara muat','Kelas jalan']
    penindakan_list = ['Peringatan','Tilang','Kepolisian','Tilang uppkb lain']
    penanganan_list = ['Penundaan perjalanan','Transfer muatan','Penandaan kelebihan dimensi','Putar balik', 'Penyesuaian persyaratan teknis','Penyesuaian tata cara muat'] 
    if pd.Series(rapor_filter_bulan[:-1]).isin([filter_bulan]).any():
        df_rapor = rapor_kab_masuk(filter_uppkb, filter_bulan)
    else:
        df_rapor = rapor_kab_masuk_all(filter_uppkb)  
    if filter_penin_penang == 'Penindakan':
        if pd.Series(rapor_filter_bulan[:-1]).isin([filter_bulan]).any():
            sss1 = rapor_kab_masuk(filter_uppkb, filter_bulan)
            sss1 = sss1[sss1['Attribute'].isin(penindakan_list)]
            result = pp_viz(sss1) 
        else:
            sss2 = rapor_kab_masuk_all(filter_uppkb)
            sss2 = sss2[sss2['Attribute'].isin(penindakan_list)]
            result = pp_viz(sss2) 
    elif filter_penin_penang == 'Penanganan':
        if pd.Series(rapor_filter_bulan[:-1]).isin([filter_bulan]).any():
            sss3 = rapor_kab_masuk(filter_uppkb,filter_bulan)
            sss3 = sss3[sss3['Attribute'].isin(penanganan_list)]
            result = pp_viz(sss3)
        else:
            sss4 = rapor_kab_masuk_all(filter_uppkb)
            sss4 = sss4[sss4['Attribute'].isin(penanganan_list)]
            result = pp_viz(sss4)  
    else:
        if pd.Series(rapor_filter_bulan[:-1]).isin([filter_bulan]).any():
            sss5 = rapor_kab_masuk(filter_uppkb,filter_bulan)
            sss5 = sss5[sss5['Attribute'].isin(pelanggaran_list)]
            result = pp_viz(sss5) 
        else:
            sss6 = rapor_kab_masuk_all(filter_uppkb)
            sss6 = sss6[sss6['Attribute'].isin(pelanggaran_list)]
            result = pp_viz(sss6)  
    rapor_pie = go.Figure(data=[go.Pie(labels=df_rapor.iloc[1:3]['Attribute'],values=df_rapor.iloc[1:3]['Jumlah KAB'])])  
    rapor_pie.update_traces(hoverinfo= 'skip',textinfo='value+percent', textfont_size=10,marker=dict(colors=color_pie))
    rapor_pie.update_layout(legend=dict( orientation="h", yanchor="bottom",y=-0.2, xanchor="center",x=0.5 ), autosize=True,height=280,margin=dict(l=0,r=0,b=20,t=0,pad=0))
    if b__ == 'all month': 
        df_bar_kat_pel = df_rapor.copy()
        df_bar_kat_pel = df_bar_kat_pel[df_bar_kat_pel['Attribute'].isin(['>5%-20%', '21%-40%', '41%-60%', '61%-80%', '81%-100%','>100%'])].rename(columns={'Attribute':'Kategori %','Jumlah KAB':'Jumlah'})
        df_bar_kat_pel['Persentase'] =[ round((i/df_bar_kat_pel['Jumlah'].sum())* 100,2)  for i in df_bar_kat_pel['Jumlah'] ]
    else :
        df_bar_kat_pel = df_rapor.copy() 
        df_bar_kat_pel = df_bar_kat_pel[df_bar_kat_pel['Attribute'].isin(['>5%-20%', '21%-40%', '41%-60%', '61%-80%', '81%-100%','>100%'])].rename(columns={'Attribute':'Kategori %','Jumlah KAB':'Jumlah'})
        df_bar_kat_pel['Persentase'] =[ round((i/df_bar_kat_pel['Jumlah'].sum())* 100,2)  for i in df_bar_kat_pel['Jumlah'] ]
    bar_kat_pel = go.Figure(go.Bar(x=df_bar_kat_pel['Jumlah'],y=df_bar_kat_pel['Kategori %'],orientation='h', text=[ '{a} (%{b})'.format(a=str(int(i)), b=j) for i,j in zip(df_bar_kat_pel['Jumlah'],df_bar_kat_pel['Persentase'])],textposition='outside',marker_color= color_pie[0])) 
    bar_kat_pel.update_traces(hoverinfo='skip')
    bar_kat_pel.update_layout(autosize=True, height=280, margin=dict(l=0,r=0,b=0,t=0,pad=0),xaxis_range=[0,df_bar_kat_pel['Jumlah'].max() + (df_bar_kat_pel['Jumlah'].max()/2)],yaxis={'categoryorder':'total ascending'}) 
    if b__ =='all month':
        month_smm = df_sort_month[df_sort_month['month'].isin(rapor_filter_bulan[:-1])]['num'].values 
        month_smm2 = df_sort_month[df_sort_month['month'].isin(rapor_filter_bulan[:-1])]['num2'].values
        bucket1 = [] 
        for i in list(data_smm.keys()):
            bucket1.append(data_smm[i].loc[u__][0])
        t_prod_nas = round(pd.Series(bucket1).mean(),2)  
        t_imp_sop = round(data_smm[month_smm[-1]].loc[u__][1],2) 
        bucket2 = [] 
        for i in list(data_smm.keys()):
            bucket2.append(data_smm[i].loc[u__][2])
        t_kons_sop = round(pd.Series(bucket2).mean(),2) 
        t_mutu_kinerja = round(data_smm[month_smm[-1]].loc[u__][3],2)
        t_persen_cap_target = round(data_smm[month_smm[-1]].loc[u__][4],2)
        t_target_100 = round(data_smm[month_smm[-1]].loc[u__][5]) 
        t_jumlah_realisasi = data_smm[month_smm[-1]].loc[u__][6] 
    else: 
        month_smm = df_sort_month[df_sort_month['month'] == b__]['num'].values[0]  
        month_smm2 = df_sort_month[df_sort_month['month'] == b__]['num2'].values[0]  
        bucket1 = [] 
        for i in list(data_smm.keys())[:month_smm2]: 
            bucket1.append(data_smm[i].loc[u__][0])
        t_prod_nas = round(pd.Series(bucket1).mean(),2) 
        t_imp_sop = round(data_smm[month_smm].loc[u__][1],2) 
        bucket2 = [] 
        for i in list(data_smm.keys())[:month_smm2]:
            bucket2.append(data_smm[i].loc[u__][2])
        t_kons_sop = round(pd.Series(bucket2).mean(),2) 
        t_mutu_kinerja = round(data_smm[month_smm].loc[u__][3],2)
        t_persen_cap_target = round(data_smm[month_smm].loc[u__][4],2) 
        t_target_100 = round(data_smm[month_smm].loc[u__][5]) 
        t_jumlah_realisasi = data_smm[month_smm].loc[u__][6]  
    df_smm = pd.DataFrame({'items': ['Rekapitulasi Produktivitas','implementtasi SOP', 'Rekaptulasi Konsistensi SOP','Penilaian Kinerja','Target Pengawasan Ideal','Realisasi Pengawasan'], 'numbers':[t_prod_nas,t_imp_sop,t_kons_sop, t_mutu_kinerja,t_target_100,t_jumlah_realisasi]}) 
    color_array = []
    for i in range(df_smm.shape[0]):
        if i % 2:
            color_array.append(color_pie[0])
        else:
            color_array.append(color_pie[1])
    tabel_smm = go.Figure(data=[go.Table(header=dict( values=['Items', 'Output'],line_color=color_pie[2], fill_color= color_pie[2],align='left',font=dict(color='white', size=14),height=30),cells=dict( values=[df_smm['items'],df_smm['numbers'],],line_color= [color_pie[0]],fill_color = [color_array*df_smm.shape[1]],align='left',font=dict(color=color_pie[2], size=14),height=30 )) ]).update_layout(autosize=True,height=280,margin=dict(l=0,r=0,b=0,t=0,pad=5)) 
    if b__ =='all month':
        df_peru = df_company2["{}".format(filter_uppkb.upper())]
    else:
        df_peru = df_company["{}".format(filter_uppkb.upper())][df_sort_month[df_sort_month['month']==filter_bulan].index[0]]
    color_array = []
    for i in range(df_peru.shape[0]):
        if i % 2:
            color_array.append(color_pie[0])
        else:
            color_array.append(color_pie[1])
    tabel_perusahaan = go.Figure(data=[go.Table(header=dict(values=['Nama Perusahaan', 'Jumlah'],line_color=color_pie[2], fill_color= color_pie[2],align='left',font=dict(color='white', size=14),height=30), cells=dict( values=[ df_peru['Nama Grup'],df_peru['Jumlah']],line_color= [color_pie[0]],fill_color= [color_array*df_peru.shape[1]], align='left',font=dict(color=color_pie[2], size=14),height=30))]).update_layout(autosize=True,height=280,margin=dict(l=0,r=0,b=0,t=0,pad=5)) 
    df_radar = df_7_indicator[(df_7_indicator['UPPKB'] == filter_uppkb)]
    radar_plot = go.Figure( data=[ go.Scatterpolar(r=df_radar['Hasil'].values, theta=df_radar['Indikator Penilaian'].values, fill='toself', fillcolor=color_pie[0],line=dict(color=color_pie[0]),opacity=1)],  layout=go.Layout(polar={'radialaxis': {'visible': True}},showlegend=True)).update_traces(fill='toself').update_layout(showlegend=False,autosize=True, height=280,margin=dict(l=0,r=0,b=0,t=0,pad=0))
    df_kab_peng = df_t3[filter_uppkb]
    fig_kab_peng = make_subplots(specs=[[{"secondary_y": True}]])
    fig_kab_peng.add_trace( go.Scatter(x=list(range(10,(df_kab_peng.shape[0]*10)+10,10)), y=df_kab_peng['KAB TERDATA/ BULAN '], name="KAB Terdata",text=df_kab_peng['KAB TERDATA/ BULAN '].astype(str),textposition="top center",mode='lines+markers+text',marker_color= '#FFCF96',marker=dict(size=8),line=dict(width=3)),secondary_y=False)
    fig_kab_peng.add_trace( go.Bar(x=list(range(10,(df_kab_peng.shape[0]*10)+10,10)), y=df_kab_peng['% PENGAWASAN'], name="Persentase Pengawasan", text=df_kab_peng['% PENGAWASAN'].astype(str) + ' %', opacity=0.2,marker_color= 'blue',),secondary_y=True)
    fig_kab_peng.update_layout( xaxis = dict( tickmode = 'array', tickvals = list(range(10,(df_kab_peng.shape[0]*10)+10,10)),ticktext =df_t3['Datae']['Bulan']),yaxis2=dict(range=[0,df_kab_peng['% PENGAWASAN'].max()*3]),legend=dict( orientation="h", yanchor="bottom",y=1, xanchor="center",x=0.5 ),autosize=True,height=400,margin=dict(l=0,r=0,b=0,t=00,pad=10))
    fig_kab_peng.update_xaxes(title_text="Bulan")
    fig_kab_peng.update_yaxes(title_text="<b>KAB Terdata</b>", secondary_y=False)
    fig_kab_peng.update_yaxes(title_text="<b>Persentase Pengawasan</b>", secondary_y=True)
    df_eval_uppkb = df_eval[(df_eval['Uppkb']==filter_uppkb)].reset_index(drop=True)
    eval_month =df_eval_uppkb['Bulan']
    eval_col_1 = ['Melanggar','Peringatan','Tilang','Kepolisian','Penundaan perjalanan','Putar balik','Transfer muatan','Penandaan kelebihan dimensi']
    eval_col_2 = ['Melanggar','Peringatan','Tilang','Kepolisian','Penundaan Perjalanan','Putar Balik','Transfer Muatan','Penandaan Kelebihan Dimensi']
    color_eval = ['#FF8080','#FFCF96','#F6FDC3','#CDFAD5','#8EACCD','#E9B384','#BEADFA','#ECE3CE']
    fig_eval = make_subplots(specs=[[{"secondary_y": True}]])
    for i,j,k in zip(eval_col_1,eval_col_2,color_eval):
        fig_eval.add_trace(go.Bar(name=j, x=eval_month, y=df_eval_uppkb[i],marker_color= k),secondary_y=False)
    fig_eval.add_trace( go.Scatter(x=eval_month, y=df_eval_uppkb['Persentase'], name="Persentase",text=df_eval_uppkb['Persentase'].astype(str) + ' %',textposition="top center",mode='lines+markers+text',),secondary_y=True)
    fig_eval.update_layout(barmode='group',legend=dict( orientation="h", yanchor="bottom",y=1, xanchor="center", x=0.5  ),yaxis2=dict(range=[0,df_eval_uppkb['Persentase'].max()+50]),autosize=True,height=400,margin=dict(l=0,r=0,b=0,t=00,pad=10))
    fig_eval.update_xaxes(title_text="Bulan")
    fig_eval.update_yaxes(title_text="<b>KAB</b>", secondary_y=False)
    fig_eval.update_yaxes(title_text="<b>Persentase</b>", secondary_y=True)
    df_eval_table = df_eval[df_eval['Uppkb'] == u__] 
    df_eval_table__= df_eval_table.T.iloc[2:].rename(columns=df_eval_table['Bulan']).reset_index().rename(columns={'index':''})
    df_eval_table__[''] = ['Kend. Masuk', 'Melanggar','Peringatan','Tilang','Kepolisian','Penundaan Perjalanan','Putar Balik','Transfer Muatan', 'Penandaan Kelebihan Dimensi', 'Persentase' ]
    color_eval_ = ['#D8D9DA','#FF8080','#FFCF96','#F6FDC3','#CDFAD5','#8EACCD','#E9B384','#BEADFA','#ECE3CE','red']
    color_map = {} 
    for i,j in zip(df_eval_table__[''], color_eval_):
        color_map.update({i:j}) 
    df_eval_table__['color'] = df_eval_table__[''].map(color_map)
    fill_color = [] 
    for i in df_eval_table__.columns:
        if i != '': 
            fill_color.append(['#e6f2fd']*len(df_eval_table__))
        else:
            fill_color.append(df_eval_table__["color"].to_list()) 
    viz_table_eval = go.Figure(data=[go.Table(header=dict( values=df_eval_table__.columns[:-1],line_color=color_pie[2], fill_color= color_pie[2], align='left',font=dict(color='white', size=14),height=30), cells=dict(values=[df_eval_table__[i] for i in df_eval_table__.iloc[:,:-1] ],line_color= [color_pie[0]],align='left',font=dict(color=color_pie[2], size=14),height=30,fill_color=fill_color )) ]).update_layout(autosize=True,height=280,margin=dict(l=10,r=10,b=10,t=10,pad=10)) 
    return 'Rapor Kinerja UPPKB {}'.format(filter_uppkb), rapor_pie, 'Resume {}'.format(filter_penin_penang), result, bar_kat_pel,tabel_perusahaan,tabel_smm, radar_plot,fig_eval, viz_table_eval,fig_kab_peng
@app.callback(Output("rekapitulasi-content", "children"),Input("url", "pathname"))
def render_page_content(pathname):
    if pathname == "/":
        return [html.Br(),lay_main_title_rekapitulasi,title_indicator,combined_line,html.Br(),title_univariate,combined_sub_title_univariate,combined_univariate,html.Br(),title_bivariate,correlation,html.Br(),title_multivariate,plot_3d,title_stat_test,table_wilcoxon,html.Br()]
    elif pathname == "/forecasting":
        return [html.Br(),lay_main_title_forecasting,title_forecasting_melanggar,Forecasting_component,title_komoditi, muatan_component ]
    elif pathname == '/raport':
        return [html.Br(),lay_main_title_rapor,component_filter_rapor,html.Br(),wrapper_title_1,wrapper_main_1,html.Br(),wrapper_title_2,wrapper_main_2,html.Br(),title_eval,component_eval,html.Br(),title_kab_peng,combined_tp,html.Br()]
@app.callback(Output("offcanvas", "is_open"),Input("open-offcanvas", "n_clicks"),[State("offcanvas", "is_open")])
def toggle_offcanvas(n1, is_open):
    if n1:
        return not is_open 
    return is_open  
if __name__ == "__main__":
    app.run_server(host='localhost',port=8064) 


C:\Users\Lenovo 16\AppData\Local\Temp\ipykernel_1620\3953955214.py:70: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

c:\Users\Lenovo 16\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning:

'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.

C:\Users\Lenovo 16\AppData\Local\Temp\ipykernel_1620\3953955214.py:531: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

Dash app running on http://localhost:8066/


C:\Users\Lenovo 16\AppData\Local\Temp\ipykernel_1620\3953955214.py:1030: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\Lenovo 16\AppData\Local\Temp\ipykernel_1620\3953955214.py:1043: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [ ]:
print(data_smm)

{'01':                         PRODUKTIFITAS Nasional  IMP. SOP Konsistensi SOP  \
UPPKB                                                                      
Dolok Estate Lima Puluh               0.583944  0.983871        0.914636   
Lubuk Selasih                         0.932333  0.983871        0.968437   
Sungai Langsat                        0.814472         1        0.929942   
Tanjung Balik                         0.857991         1        0.967445   
Muara Lembu                           0.942602  0.967742        0.957419   
Muara Tembesi                         0.689453         1        0.854916   
Merapi                                0.747747         1         0.78508   
Cibaragalan                           1.336947  0.983871        0.983199   
Tomo                                  0.933156         1               1   
Sarang                                1.036724         1        0.951567   
Subah                                 1.191082         1        0.962274   
Taman

: 

: 

: 